# VRPTW

In [2]:
import time
import csv
import numpy as np
import math
import random
import copy
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
E:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
# parameter value setting
run_time = time.strftime("%m%d_%H%M", time.localtime())
veh_spd_kmh = 50  # supposed vehicle speed in km/h
veh_spd = veh_spd_kmh * 1000 / 60.  # supposed vehicle speed in m/min(50km/h, 833m/min)
vehicle_type_dict = {2: '2T', 3: '3T', 5: '5T'}
small_veh = [2, 2, 10, 540, 0.004, 200]  # [veh_type, load weight, effective volume, trvl_dist, trs_cost, fix_cost]
medium_veh = [3, 3, 11, 540, 0.005, 200]  # [veh_type, load weight, effective volume, trvl_dist, trs_cost, fix_cost]
large_veh = [5, 5, 17, 540, 0.006, 300]  # [veh_type, load weight, effective volume, trvl_time, trs_cost, fix_cost]
bskt_vol = 0.65 * 0.44 * 0.16  # volume of fresh basket
trsf_vol = 0.56 * 0.39 * 0.32  # volume of transfer box
milk_vol = 0.001  # volume of both fresh milk and skim milk is 1 litre
paper_bskt = 1.2 * 1 * 0.6  # volume of paper basket
oprt_t = 25  # operation time at each store
start_t = 0  # earliest start time of a vehicle
small_veh_cnt = 60  # setup(initial) small vehicle number
medium_veh_cnt = 39  # setup(initial) medium vehicle number
alp, bet, gam = 0.6, 0.2, 0.2  # weight of t_ij, h_ij and v_ij of Time-oriented Nearest-Neighbor  

In [4]:
def read_data():
    pnt = 664
    num_id = {}  # store number: [id, name, address]
    id_num = {}  # id: number
    loc = np.zeros([pnt, 2])  # longitude and latitude
    demd = {}  # store demand: [bskt_num, trsf_num, fresh_milk, skim_milk]
    time_zone = {}  # store service time zone: [earliest start time, latest start time]

    with open(r'G:\CDO\海上风电场\LNS\JD-GOC-VRP-master\data\inputnode_1_1601.csv', 'rU') as fr:
        reader = csv.reader(fr)
        next(reader)
        itr = 0
        for v in reader:
            itr += 1
            if itr > pnt:
                break
            store_num = int(v[0])
            num_id[store_num] = v[1:4]
            id_num[v[1]] = store_num
            loc[store_num] = v[4:6]
            demd[store_num] = [int(v[8]), int(v[9]), int(v[10]), int(v[11]), int(v[12])]
            first_t = v[6].split(':')
            last_t = v[7].split(':')
            time_zone[store_num] = [int(first_t[0])*60 + int(first_t[1]), int(last_t[0])*60 + int(last_t[1])]

    return num_id, id_num, loc, demd, time_zone 

In [5]:
def earth_dist(lng_lat):
    '''给定一个点数组(经度和纬度)，该函数将返回一个距离矩阵'''
    r = 6371.004 * 1000  # radius of earth(6371km)
    ln = len(lng_lat)  # number of points
    dist_matr = np.zeros([ln, ln])
    time_matr = np.zeros([ln, ln])
    for i in range(ln):
        for j in range(i + 1, ln):
            ax = math.pi * lng_lat[i, 0] / 180.0  # 转换角度到弧度
            ay = math.pi * lng_lat[i, 1] / 180.0
            bx = math.pi * lng_lat[j, 0] / 180.0
            by = math.pi * lng_lat[j, 1] / 180.0
            dist = r * math.acos(math.sin(ay) * math.sin(by) + math.cos(ay) * math.cos(by) * math.cos(bx - ax))
            dist_matr[i, j] = round(dist)
            dist_matr[j, i] = dist_matr[i, j]
            time_matr[i, j] = dist_matr[i, j] / veh_spd  # suppose the vehicle driving speed is 50km/h(833.3m/min)
            time_matr[j, i] = time_matr[i, j]

    return dist_matr, time_matr  

In [6]:
class GetInitial(object):
    """得到初始解。"""

    def __init__(self):
        self.a = 5

    def time_nn(self, on_way_time, curr_cust, remain_list, used_resource, rout_len, vehicle_type):
        """给定车辆及其当前来访的客户，返回下一个来访的客户.
        
        The closeness between customer i and j: C_ij = alp1*t_ij + alp2*h_ij + alp3*v_ij,
        t_ij: travel time between i and j (distance);
        h_ij = b_j - (b_i + s_i), b is start service time, and s is service time (idle time);
        v_ij = l_j - (b_i + s_i + t_ij), l is the latest admissible service time, t_ij is the travel time (urgency)
        The low value of C_ij, the better.
        """
        if vehicle_type == 2:
            veh_cap = small_veh
        elif vehicle_type == 3:
            veh_cap = medium_veh
        else:
            veh_cap = large_veh
        real_wait_time = 0  # 测试所有可能存储后的最后等待时间
        real_vst_cust = -1  # 测试所有可能的商店后的最后访问商店
        visit_cust = [-1, 100000, 600000, 10000]  # [cust_id, next_start, distance, closeness]
        if rout_len - 1 < 50:  # 车辆访问的最大商店数
            for cust in remain_list:
                # print('checking customer: ', cust)
                if (used_resource[0] + num_demd[cust][0] * bskt_vol + num_demd[cust][1] * trsf_vol + (num_demd[cust][2] +
                        num_demd[cust][3]) * milk_vol + num_demd[cust][4] * paper_bskt) > veh_cap[2]:
                    # print('run out of effective volume')
                    continue  # volume overload
                # elif dist_mat[curr_cust, cust] + dist_mat[cust, 0] > veh_cap[3] - used_resource[3]:
                #     print('run out of distance')
                #     continue
                elif used_resource[2] + time_mat[curr_cust, cust] > num_timez[cust][1]:
                    # print('late than last receive time')
                    continue  # 不能在最后一次接收时间之前到达
                elif time_mat[curr_cust, cust] + oprt_t + time_mat[cust, 0] > veh_cap[3] - on_way_time:
                    # print('run out of work time')
                    continue
                elif (curr_cust > 0 and used_resource[2] + time_mat[curr_cust, cust] < num_timez[cust][0] and
                        num_timez[cust][0] - used_resource[2] + oprt_t + time_mat[cust, 0] > veh_cap[3] - on_way_time):
                    # print('run out of work time - with waiting time')
                    continue
                else:
                    wait_time = num_timez[cust][0] - (used_resource[2] + time_mat[curr_cust, cust])

                    if wait_time < 0:
                        next_start = used_resource[2] + time_mat[curr_cust, cust]
                        h_ij = time_mat[curr_cust, cust]
                    else:  # 提前到达
                        next_start = num_timez[cust][0]
                        if curr_cust == 0:
                            h_ij = time_mat[curr_cust, cust]
                            wait_time = 0   # 仓库出发特殊情况
                        else:
                            h_ij = next_start - used_resource[2]
                    v_ij = num_timez[cust][1] - (used_resource[2] + time_mat[curr_cust, cust])
                    close_ij = alp * time_mat[curr_cust, cust] + bet * h_ij + gam * v_ij  # i和j之间的紧密关系
                    # print(curr_cust, cust, close_ij)
                    if close_ij < visit_cust[3]:
                        real_wait_time = wait_time
                        real_vst_cust = cust
                        visit_cust[0] = cust
                        visit_cust[1] = next_start
                        visit_cust[2] = dist_mat[curr_cust, cust]
                        visit_cust[3] = close_ij
                    else:
                        continue


        if visit_cust[0] == -1:  # 无客户来访
            visit_cust[0] = 0
            visit_cust[1] = used_resource[-1] + time_mat[curr_cust, 0]
            on_way_time += time_mat[curr_cust, 0]
        else:
            # print(curr_cust, real_vst_cust, real_wait_time)
            if real_wait_time <= 0:
                on_way_time += (oprt_t + time_mat[curr_cust, real_vst_cust])
            else:
                on_way_time += (oprt_t + real_wait_time + time_mat[curr_cust, real_vst_cust])

        return visit_cust, on_way_time


    def greedy_initial(self):
        """
        基于Solomon提出的面向时间的最近邻启发式生成初始解.
        """
        sol = []  # [[0;2;5;0;4;6;0],[],...]
        sol_veh_type = []  # 对应车型的解决方案
        route_way_time = []

        to_vist = [i+1 for i in range(store_num - 1)]  # [1,5,8,...]
        itr = 0

        while len(to_vist) > 0 and itr < 500:
            itr += 1

            if itr <= small_veh_cnt:
                vehicle_type0 = 2
            elif itr <= small_veh_cnt + medium_veh_cnt:
                vehicle_type0 = 3
            else:
                vehicle_type0 = 5

            sol_veh_type.append(vehicle_type0)

            used_res = [0, 0, 0, 0]  # 车辆的使用量、行驶时间、离开时间、行驶距离
            veh_rout = [0]

            # print '\nA new vehicle will be used.'
            way_time = 0  # 到店出行时间+到店等待时间+到店运营时间
            while True:
                curr_cust = veh_rout[-1]

                next_one, way_time = self.time_nn(way_time, curr_cust, to_vist, used_res, len(veh_rout), vehicle_type0)
                next_cust, next_start = next_one[0], next_one[1]
                # print('next start', next_cust, next_start)
                if next_cust == 0:  # next visiting customer is depot
                    # print 'Get back to the depot, and ready for a new round.'
                    veh_rout.append(next_cust)
                    break

                else:  # next visiting customer is a store
                    used_res[0] += (num_demd[next_cust][0] * bskt_vol + num_demd[next_cust][1] * trsf_vol + (num_demd[next_cust][2] + \
                                    num_demd[next_cust][3]) * milk_vol + num_demd[next_cust][4] * paper_bskt)
                    used_res[2] = (next_start + oprt_t)
                    used_res[3] += dist_mat[curr_cust, next_cust]


                veh_rout.append(next_cust)
                # print 'Vehicle used resource: ', used_res
                to_vist.remove(next_cust)

            sol.append(veh_rout)
            route_way_time.append(way_time)

            # print 'Last point 0 earliest leave time: ', int(used_res[-1]) / 60, ':', int(used_res[-1]) % 60
            # print 'Route %s is: ' % itr, veh_rout
            print('*'*10, 'Iteration:', itr, '*'*10)


        if len(to_vist) > 0:
            print('number of stores remained: ', len(to_vist))

        return sol, sol_veh_type, route_way_time   

In [7]:
class OutputFormat(object):
    """以特定的格式打印结果:路线详情，路线摘要，计划摘要"""

    def __init__(self):
        self.c = 4


    def print_result(self, solution, vehicle_type, if_write):
        """给定列表中保存的解决方案，计算解决方案的总成本.
        将解决方案以所需格式写入本地."""

        result = [['Vehicle_ID', 'Vehicle_type', 'Route', 'Leave_Time', 'Back_Time', 'Work_Time', 'Distance',
                   'Load_Volume', 'Wait_Time', 'Fixed_Cost', 'Travel_Cost', 'Total_Cost']]
        total_dist = 0
        total_cost = 0
        for k, veh in enumerate(solution):
            if len(veh) == 2:
                continue

            if vehicle_type[k] == 2:
                trans0 = small_veh[4]
                fix0 = small_veh[5]
            elif vehicle_type[k] == 3:
                trans0 = medium_veh[4]
                fix0 = medium_veh[5]
            else:
                trans0 = large_veh[4]
                fix0 = large_veh[5]

            total_cost += fix0
            departt = check_violation(veh, vehicle_type[k])[3]

            trvl_dist = 0
            veh_load_vol = 0
            wait_time = 0

            # 获取输出格式
            route = [0] * len(result[0])
            route[0] = k + 1  # vehicle name
            route[1] = vehicle_type_dict[vehicle_type[k]]  # vehicle type
            route_ele = []
            for ele in veh:
                if ele == 0:
                    route_ele.append(str(ele))
                else:
                    route_ele.append(num_id[ele][0])
            route[2] = '-'.join(route_ele)  # route

            trvl_dist += (dist_mat[0, veh[1]] + dist_mat[veh[-2], 0])
            veh_load_vol += (num_demd[veh[1]][0] * bskt_vol + num_demd[veh[1]][1] * trsf_vol + (num_demd[veh[1]][2] +
                             num_demd[veh[1]][3]) * milk_vol + num_demd[veh[1]][4] * paper_bskt)
            if departt / 60. < 24.:
                out_time = int(departt)
            else:
                out_time = int(departt - 24 * 60)
            route[3] = str(out_time // 60) + ':' + str(out_time % 60).zfill(2)
            t = departt + time_mat[0, veh[1]] + oprt_t
            for i in range(2, len(veh) - 1):  # can not wait at the first 2 points
                trvl_dist += dist_mat[veh[i - 1], veh[i]]
                veh_load_vol += (num_demd[veh[i]][0] * bskt_vol + num_demd[veh[i]][1] * trsf_vol + (num_demd[veh[i]][2] +
                                 num_demd[veh[i]][3]) * milk_vol + num_demd[veh[i]][4] * paper_bskt)
                wait_t = num_timez[veh[i]][0] - (t + time_mat[veh[i - 1], veh[i]])
                if wait_t > 0 + 1e-5:
                    # print veh[i-1], veh[i], wait_t
                    wait_time += wait_t
                    t = num_timez[veh[i]][0] + oprt_t
                else:
                    t += (time_mat[veh[i - 1], veh[i]] + oprt_t)
            if t + time_mat[veh[-2], 0] < 24. * 60:
                in_time = int(t + time_mat[veh[-2], 0])
            else:
                in_time = int(t + time_mat[veh[-2], 0] - 24 * 60)

            route[4] = str(in_time // 60) + ':' + str(in_time % 60).zfill(2)  # vehicle back time
            route[5] = round((t + time_mat[veh[-2], 0] - departt) / 60., 1)
            route[6] = round(trvl_dist / 1000., 2)  # total distance
            route[7] = veh_load_vol  # vehicle load volume
            route[8] = wait_time  # vehicle wait time
            route[9] = fix0  # vehicle fixed cost
            route[10] = round(trvl_dist * trans0, 2)  # vehicle travel cost
            route[11] = route[9] + route[10]  # total cost

            total_cost += trvl_dist * trans0
            result.append(route)
            # print route
            total_dist += route[6]
            # print 'Last leave time: ', int(t) / 60, ':', int(t) % 60
            # print 'total distances: ', route[5]

        if if_write:
            run_time = time.strftime("%m%d_%H%M", time.localtime())
            with open(r'G:\CDO\海上风电场\LNS\JD-GOC-VRP-master\data\Route_Plan_%s.csv' % run_time, 'w', newline='') as fw:
                writer = csv.writer(fw)
                for v in result:
                    writer.writerow(v)

        return round(total_cost, 2)


    def print_route_detail(self, solution, vehicle_type, if_write):
        """给定列表中保存的解决方案，计算解决方案的总成本.
        将解决方案以所需格式写入本地."""

        result = [[
            '线路编号',
            '门店编码',
            '门店名称',
            '门店地址',
            '经度',
            '纬度',
            '车型',
            '额定体积/m3',
            '额定重量/t',
            '到达时间',
            '离开时间',
            '行驶距离/km',
            '累计行驶距离km',
            '行驶时间/min',
            '卸货时间/min',
            '累计工作时间/h',
            '鲜食篮总数',
            '周转箱个数',
            '新绿园鲜奶980ML（罐）',
            '新绿园脱脂牛奶980ML（罐）',
            '纸箱个数',
            '卸货体积',
            '卸货重量']]

        total_dist = 0
        for k, veh in enumerate(solution):
            if vehicle_type[k] == 2:
                trans0 = small_veh[4]
                veh_param = small_veh

            elif vehicle_type[k] == 3:
                trans0 = medium_veh[4]
                veh_param = medium_veh

            else:
                trans0 = large_veh[4]
                veh_param = large_veh


            departt = check_violation(veh, vehicle_type[k])[3]
            t = departt

            trvl_dist = 0
            veh_load_vol = 0
            wait_time = 0

            veh_load_vol += (num_demd[veh[1]][0] * bskt_vol + num_demd[veh[1]][1] * trsf_vol + (num_demd[veh[1]][2] +
                             num_demd[veh[1]][3]) * milk_vol + num_demd[veh[1]][4] * paper_bskt)
            if departt / 60. < 24.:
                out_time = int(math.ceil(departt))
            else:
                out_time = int(math.ceil(departt - 24 * 60))

            # get the output format
            store = [0] * len(result[0])
            store[0] = k + 1  # 线路序号
            store[1] = num_id[0][0]  # 门店编号
            store[2] = num_id[0][1]  # 门店名称
            store[3] = num_id[0][2]  # 门店地址
            store[4] = loc[0][0]  # 经度
            store[5] = loc[0][1]  # 纬度
            store[6] = vehicle_type_dict[vehicle_type[k]]  # 车型
            store[7] = veh_param[2]  # 额定体积
            store[8] = veh_param[1]  # 额定重量
            store[9] = str(out_time // 60) + ':' + str(out_time % 60).zfill(2)  # 到达时间
            store[10] = str(out_time // 60) + ':' + str(out_time % 60).zfill(2)  # 离开时间
            store[11] = 0  # 行驶距离
            store[12] = 0  # 累计行驶距离
            store[13] = 0  # 行驶时间
            store[14] = 0  # 卸货时间
            store[15] = 0  # 累计工作时间
            store[16] = 0  # 鲜食篮件数
            store[17] = 0  # 周转箱个数
            store[18] = 0  # 新绿园鲜奶
            store[19] = 0  # 新绿园脱脂牛奶
            store[20] = 0  # 纸箱
            store[21] = 0  # 卸货体积
            store[22] = 0  # 卸货重量

            store0 = copy.deepcopy(store)
            result.append(store0)

            # t = departt + time_mat[0, veh[1]] + oprt_t  # t is the leaving time
            for i in range(1, len(veh)-1):  # can not wait at the first 2 points
                store[1] = num_id[veh[i]][0]
                store[2] = num_id[veh[i]][1]
                store[3] = num_id[veh[i]][2]
                store[4] = loc[veh[i]][0]
                store[5] = loc[veh[i]][1]
                arr_time = t + time_mat[veh[i-1], veh[i]]
                if arr_time / 60. < 24.:
                    in_time = int(math.ceil(arr_time))
                else:
                    in_time = int(math.ceil(arr_time - 24 * 60))

                trvl_dist += dist_mat[veh[i-1], veh[i]]
                veh_load_vol += (num_demd[veh[i]][0] * bskt_vol + num_demd[veh[i]][1] * trsf_vol + (num_demd[veh[i]][2] +
                                 num_demd[veh[i]][3]) * milk_vol + num_demd[veh[i]][4] * paper_bskt)
                wait_t = num_timez[veh[i]][0] - (t + time_mat[veh[i-1], veh[i]])
                if wait_t > 0 + 1e-5:
                    # t is the leaving time
                    wait_time += wait_t
                    t = num_timez[veh[i]][0] + oprt_t
                else:
                    t += (time_mat[veh[i - 1], veh[i]] + oprt_t)
                if t < 24. * 60:
                    out_time = int(math.ceil(t))
                else:
                    out_time = int(math.ceil(t - 24 * 60))

                store[9] = str(in_time // 60) + ':' + str(in_time % 60).zfill(2)  # 到达时间
                store[10] = str(out_time // 60) + ':' + str(out_time % 60).zfill(2)  # 离开时间
                store[11] = round(dist_mat[veh[i-1], veh[i]] / 1000., 2)  # 行驶距离
                store[12] = round(trvl_dist / 1000., 2)  # 累计行驶距离
                store[13] = round(time_mat[veh[i-1], veh[i]], 1)  # 行驶时间
                store[14] = oprt_t
                store[15] = round((t - departt) / 60., 2)  # 累计工作时间
                store[16] = num_demd[veh[i]][0]  # 鲜食篮件数
                store[17] = num_demd[veh[i]][1]  # 周转箱个数
                store[18] = num_demd[veh[i]][2]  # 新绿园鲜奶
                store[19] = num_demd[veh[i]][3]  # 新绿园脱脂牛奶
                store[20] = num_demd[veh[i]][4]  # 纸箱
                store[21] = (num_demd[veh[i]][0] * bskt_vol + num_demd[veh[i]][1] * trsf_vol + (num_demd[veh[i]][2] +
                            num_demd[veh[i]][3]) * milk_vol + num_demd[veh[i]][4] * paper_bskt)  # 卸货体积
                store[22] = 0  # 卸货重量

                store0 = copy.deepcopy(store)
                result.append(store0)
                # print(result[-1])

            store[1] = num_id[0][0]  # 门店编号
            store[2] = num_id[0][1]  # 门店名称
            store[3] = num_id[0][2]  # 门店地址
            store[4] = loc[0][0]  # 经度
            store[5] = loc[0][1]  # 纬度
            arr_time = t + time_mat[veh[-2], 0]
            if arr_time / 60. < 24.:
                in_time = int(math.ceil(arr_time))
            else:
                in_time = int(math.ceil(arr_time - 24 * 60))
            store[9] = str(in_time // 60) + ':' + str(in_time % 60).zfill(2)  # 到达时间
            store[10] = str(in_time // 60) + ':' + str(in_time % 60).zfill(2)  # 离开时间
            store[11] = round(dist_mat[veh[-2], 0] / 1000., 2)  # 行驶距离
            store[12] = round((trvl_dist + dist_mat[veh[-2], 0]) / 1000., 2)  # 累计行驶距离
            store[13] = round(time_mat[veh[-2], 0], 1)  # 行驶时间
            store[14] = 0  # 卸货时间
            store[15] = round((t - departt + time_mat[veh[-2], 0]) / 60., 2)  # 累计工作时间
            store[16] = 0  # 鲜食篮件数
            store[17] = 0  # 周转箱个数
            store[18] = 0  # 新绿园鲜奶
            store[19] = 0  # 新绿园脱脂牛奶
            store[20] = 0  # 纸箱
            store[21] = 0  # 卸货体积
            store[22] = 0  # 卸货重量

            store0 = copy.deepcopy(store)
            result.append(store0)
            # print(result[-1])

        if if_write:
            # run_time = time.strftime("%m%d_%H%M", time.localtime())
            with open(r'C:\Bee\0Huaat\Starbucks\output\Route_Details_%s_%s.csv' % (veh_spd_kmh, run_time), 'w', newline='') as fw:
                writer = csv.writer(fw)
                for v in result:
                    # print(v)
                    writer.writerow(v)


    def print_route_summary(self, solution, vehicle_type, if_write):
        """Given the solution saved in list, calculate the total cost of the solution.
        Write the solution to local in the required format."""
        result_summary = [[
            '计划编号',
            '门店数',
            '配送总体积/m3',
            '配送总重量/t',
            '设定车速/km/h',
            '总车数',
            '总额定体积/m3',
            '总额定重量/t',
            '体积装载率/%',
            '重量装载率/%',
            '总行驶里程/km',
            '有效里程/km',
            '返空里程/km',
            '工作时间/h',
            '行驶时间/min',
            '卸货时间/min',
            '总成本/元',
            '固定成本/元',
            '运输成本/元',
            '2T车数量',
            '3T车数量',
            '5T车数量',
            '鲜食篮总数',
            '周转箱个数',
            '新绿园鲜奶980ML（罐）',
            '新绿园脱脂牛奶980ML（罐）',
            '纸箱个数']]
        summ_value = [0] * len(result_summary[0])

        result = [[
            '线路编号',
            '出发时间',
            '返回时间',
            '工作时间/h',
            '行驶总时间/min',
            '卸货总时间/min',
            '等待时间/min',
            '总行驶里程/km',
            '有效里程/km',
            '返空里程/km',
            '车型',
            '额定装载体积/m3',
            '额定装载重量/t',
            '实际装载体积/m3',
            '实际装载重量/t',
            '体积装载率/%',
            '重量装载率/%',
            '总成本/元',
            '固定成本/元',
            '运输成本/元',
            '配送门店总数',
            '门店1编号',
            '门店1名称',
            '门店2编号',
            '门店2名称',
            '门店3编号',
            '门店3名称',
            '门店4编号',
            '门店4名称',
            '门店5编号',
            '门店5名称',
            '门店6编号',
            '门店6名称',
            '门店7编号',
            '门店7名称',
            '门店8编号',
            '门店8名称',
            '门店9编号',
            '门店9名称',
            '门店10编号',
            '门店10名称',
            '门店11编号',
            '门店11名称',
            '门店12编号',
            '门店12名称',
            '门店13编号',
            '门店13名称',
            '门店14编号',
            '门店14名称',
            '门店15编号',
            '门店15名称',
            '门店16编号',
            '门店16名称',
            '门店17编号',
            '门店17名称',
            '门店18编号',
            '门店18名称',
            '门店19编号',
            '门店19名称',
            '门店20编号',
            '门店20名称']]

        total_dist = 0
        for k, veh in enumerate(solution):
            if vehicle_type[k] == 2:
                trans0 = small_veh[4]
                veh_param = small_veh
                summ_value[19] += 1
            elif vehicle_type[k] == 3:
                trans0 = medium_veh[4]
                veh_param = medium_veh
                summ_value[20] += 1
            else:
                trans0 = large_veh[4]
                veh_param = large_veh
                summ_value[21] += 1

            departt = check_violation(veh, vehicle_type[k])[3]

            trvl_dist = 0
            veh_load_vol = 0
            wait_time = 0
            trvl_time = 0

            # get the output format
            route = [0] * 21
            route[0] = k + 1  # vehicle name
            route[10] = vehicle_type_dict[vehicle_type[k]]  # 车型


            trvl_dist += (dist_mat[0, veh[1]] + dist_mat[veh[-2], 0])
            trvl_time += (time_mat[0, veh[1]] + time_mat[veh[-2], 0])
            veh_load_vol += (num_demd[veh[1]][0] * bskt_vol + num_demd[veh[1]][1] * trsf_vol + (num_demd[veh[1]][2] +
                             num_demd[veh[1]][3]) * milk_vol + num_demd[veh[1]][4] * paper_bskt)

            summ_value[22] += num_demd[veh[1]][0]
            summ_value[23] += num_demd[veh[1]][1]
            summ_value[24] += num_demd[veh[1]][2]
            summ_value[25] += num_demd[veh[1]][3]
            summ_value[26] += num_demd[veh[1]][4]

            if departt / 60. < 24.:
                out_time = int(departt)
            else:
                out_time = int(departt - 24 * 60)
            route[1] = str(out_time // 60) + ':' + str(out_time % 60).zfill(2)
            t = departt + time_mat[0, veh[1]] + oprt_t
            for i in range(2, len(veh)-1):  # can not wait at the first 2 points
                trvl_dist += dist_mat[veh[i-1], veh[i]]
                trvl_time += time_mat[veh[i-1], veh[i]]
                veh_load_vol += (num_demd[veh[i]][0] * bskt_vol + num_demd[veh[i]][1] * trsf_vol + (num_demd[veh[i]][2] +
                                 num_demd[veh[i]][3]) * milk_vol + num_demd[veh[i]][4] * paper_bskt)

                summ_value[22] += num_demd[veh[i]][0]
                summ_value[23] += num_demd[veh[i]][1]
                summ_value[24] += num_demd[veh[i]][2]
                summ_value[25] += num_demd[veh[i]][3]
                summ_value[26] += num_demd[veh[i]][4]

                wait_t = num_timez[veh[i]][0] - (t + time_mat[veh[i-1], veh[i]])
                if wait_t > 0 + 1e-5:
                    # print veh[i-1], veh[i], wait_t
                    wait_time += wait_t
                    t = num_timez[veh[i]][0] + oprt_t
                else:
                    t += (time_mat[veh[i - 1], veh[i]] + oprt_t)
            if t + time_mat[veh[-2], 0] < 24. * 60:
                in_time = int(t + time_mat[veh[-2], 0])
            else:
                in_time = int(t + time_mat[veh[-2], 0] - 24 * 60)

            route[2] = str(in_time // 60) + ':' + str(in_time % 60).zfill(2)  # 返回时间
            route[3] = round((t + time_mat[veh[-2], 0] - departt) / 60., 1)  # 工作时间
            route[4] = round(trvl_time, 1)  # 行驶时间
            route[5] = round(oprt_t * (len(veh) - 2), 1)  # 操作时间
            route[6] = wait_time
            route[7] = round(trvl_dist / 1000., 2)  # 行驶里程
            route[8] = round((trvl_dist - dist_mat[veh[-2], 0]) / 1000., 2)  # 有效里程
            route[9] = round(dist_mat[veh[-2], 0] / 1000., 2)  # 返空里程
            route[11] = veh_param[2]  # 额定体积
            route[12] = veh_param[1]  # 额定重量
            route[13] = veh_load_vol  # 实际装载体积
            route[14] = 0.  # 实际装载重量
            route[15] = round(veh_load_vol / veh_param[2] * 100, 2)  # 体积装载率
            route[16] = round(route[14] / veh_param[1] * 100, 2)  # 重量装载率
            route[18] = veh_param[-1]  # 固定成本
            route[19] = round(trvl_dist * trans0, 2)  # 运输成本
            route[17] = route[18] + route[19]  # 总成本
            route[20] = len(veh) - 2  # 配送门店总数

            for ele in veh:
                if ele != 0:
                    route.append(num_id[ele][0])
                    route.append(num_id[ele][1])


            result.append(route)
            # print route
            total_dist += route[7]
            # print 'Last leave time: ', int(t) / 60, ':', int(t) % 60
            # print 'total distances: ', route[5]

            summ_value[2] += veh_load_vol
            summ_value[3] += 0
            summ_value[4] = veh_spd_kmh
            summ_value[5] += 1
            summ_value[6] += veh_param[2]
            summ_value[7] += veh_param[1]
            summ_value[10] += round(trvl_dist / 1000., 2)
            summ_value[11] += route[8]
            summ_value[12] += route[9]
            summ_value[13] += route[3]
            summ_value[14] += route[4]
            summ_value[15] += route[5]
            summ_value[16] += route[17]
            summ_value[17] += route[18]
            summ_value[18] += route[19]


        if if_write:
            # run_time = time.strftime("%m%d_%H%M", time.localtime())
            with open(r'C:\Bee\0Huaat\Starbucks\output\Route_Summary_%s_%s.csv' % (veh_spd_kmh, run_time), 'w', newline='') as fw:
                writer = csv.writer(fw)
                for v in result:
                    writer.writerow(v)


            summ_value[0] = run_time
            summ_value[1] = store_num - 1
            summ_value[8] = round(summ_value[2] / summ_value[6] * 100, 2)
            summ_value[9] = round(summ_value[3] / summ_value[7] * 100, 2)
            result_summary.append(summ_value)
            with open(r'C:\Bee\0Huaat\Starbucks\output\Plan_Summary_%s_%s.csv' % (veh_spd_kmh, run_time), 'w', newline='') as fww:
                writer = csv.writer(fww)
                for vv in result_summary:
                    writer.writerow(vv)


        return total_dist

In [8]:
def check_violation(route, vehicle_type):
    """用给定的车辆类型检查路线是否可行，并返回检查结果和路线成本."""
    if len(route) == 2:  # [0, 0] route
        return True, 0, 0, 0
    else:
        accu_res = [0, 0, 0]  # 0-leaving time, 1-accumulated distance, 2-volume
        if vehicle_type == 2:
            veh_cap = small_veh
        elif vehicle_type == 3:
            veh_cap = medium_veh
        elif vehicle_type == 5:
            veh_cap = large_veh
        else:
            veh_cap = large_veh
            print('Input wrong vehicle type!', vehicle_type)
        # small_veh = [1, 12, 10, 400000, 0.012, 200]
        fixed_cost = veh_cap[5]
        trans_cost = 0
        # wait_cost = 0
        if time_mat[0, route[1]] < num_timez[route[1]][0]:
            accu_res[0] = num_timez[route[1]][0] - time_mat[0, route[1]]  # 车辆离开车场时间
            depart_time = accu_res[0]  # departing from depot time  离厂时间
        else:
            depart_time = 0
        for i in range(len(route) - 1):
            last_cust = route[i]
            curr_cust = route[i+1]
            # checking leaving time
            arr_time = accu_res[0] + time_mat[last_cust, curr_cust]
            if arr_time < num_timez[curr_cust][0]:
                accu_res[0] = num_timez[curr_cust][0] + oprt_t
                wait_time = num_timez[curr_cust][0] - arr_time
                # wait_cost += (wait_time / 60. * wait_cost0)
            elif arr_time <= num_timez[curr_cust][1]:
                accu_res[0] = arr_time + oprt_t
            else:
                # print('Infeasible route!(Service Time Error.)')
                return False, 1000000, 0, 0

            # checking vehicle max distance
            trans_cost += (dist_mat[last_cust, curr_cust] * veh_cap[4])

            accu_res[1] += dist_mat[last_cust, curr_cust]

            if accu_res[0] - oprt_t - depart_time > veh_cap[3]:
                # print('Infeasible route!(Max Time Error.)')
                return False, 1000000, 0, 0

            # checking vehicle max volume
            accu_res[2] += (num_demd[curr_cust][0] * bskt_vol + num_demd[curr_cust][1] * trsf_vol + (num_demd[curr_cust][2]
                            + num_demd[curr_cust][3]) * milk_vol + num_demd[curr_cust][4] * paper_bskt)

            if accu_res[2] > veh_cap[2]:
                # print('Infeasible route!(Max Weight/Volume Error.)', accu_res[2])
                return False, 1000000, 0, 0
    route_cost = fixed_cost + accu_res[1] * veh_cap[4]
    route_dist = accu_res[1]
    route_time = accu_res[0] - oprt_t - depart_time
    # print fixed_cost, trvl_cost, trvl_dist
    return True, route_cost, route_time, depart_time + 600


In [9]:
def plot_route(sol):
    px = loc[:, 0]
    py = loc[:, 1]
    plt.ion()  # 开启interactive mode 成功的关键函数
    plt.figure(1)
    plt.scatter(px, py, color='black', marker='*')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    color_list = ['b', 'g', 'pink', 'purple', 'y', 'grey', 'gold', 'violet']
    col_ind = 0
    for route in sol:
        if col_ind < len(color_list) - 1:
            route_color = color_list[col_ind]
            col_ind += 1
        else:
            route_color = color_list[col_ind]
            col_ind = 0
        for i in range(len(route)-1):
            curr_p = route[i]
            next_p = route[i+1]
            plt.plot([px[curr_p], px[next_p]], [py[curr_p], py[next_p]], color=route_color)
            plt.draw()
            plt.pause(0.1)

    # 关闭交互模式
    plt.ioff()
    plt.show()

In [10]:
def cust_loc(sol, cust):
    """获取一个客户的路线位置和客户位置."""
    cust_ind = []  # [route_loc, cust_loc]
    for i, rt in enumerate(sol):
        if cust in rt:
            cust_ind.append(i)
            cust_ind.append(rt.index(cust))
            return cust_ind

    print('Costomer not in the solution: ', cust)


In [11]:
def route_type(route):
    """给定一条路线，返回该路线的车辆类型。小型车辆第一，中型车辆第二，大型车辆最后."""
    typ = 2
    vol_accu = 0  # accumulated volume

    if len(route) <= 2:
        return typ
    else:
        for i in range(1, len(route) - 1):
            cust0 = route[i]
            vol_accu += (num_demd[cust0][0] * bskt_vol + num_demd[cust0][1] * trsf_vol + (num_demd[cust0][2] +
                         num_demd[cust0][3]) * milk_vol + num_demd[cust0][4] * paper_bskt)

    if vol_accu <= small_veh[2]:
        return 2
    elif vol_accu <= medium_veh[2]:
        return 3
    elif vol_accu <= large_veh[2]:
        return 5
    else:
        print('!!!Route is invalid: out of max volume!', route)

In [12]:
class SimulatedAnnealing(object):
    """采用模拟退火算法对初始解进行细化.
    使用操作符生成vrp领域."""

    def __init__(self):
        self.b = 6


    def shift_1_cust(self, sol_in1, cust, c_loc, curr_temp, sol_type1, sa_lns):
        """试着把一个客户搬到任何可以安置的地方，看看这样做是否能降低总成本."""

        route_ing = copy.deepcopy(sol_in1[c_loc[0]])
        route_new = route_ing
        move_to_route = c_loc[0]
        orgn_type1 = sol_type1[c_loc[0]]
        origin_cost1 = check_violation(route_ing, orgn_type1)[1]
        route_ing.remove(cust)  # move c in the current route
        new_type1 = route_type(route_ing)
        adjust_cost1 = check_violation(route_ing, new_type1)[1]
        best_cut_cost0 = -1000
        best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
        for j, rou in enumerate(sol_in1):
            orgn_type2 = sol_type1[j]
            origin_cost2 = check_violation(rou, orgn_type2)[1]
            if j == c_loc[0]:  # moving in the same route
                for k in range(1, len(route_ing)):
                    if k == c_loc[1]:
                        continue  # do not put it at the original position
                    rou_test = route_ing[:k] + [cust] + route_ing[k:]
                    if check_violation(rou_test, orgn_type2)[0]:
                        adjust_cost2 = check_violation(rou_test, orgn_type2)[1]
                        cost_cut_test = origin_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new = rou_test
                            move_to_route = j


            else:  # moving to a different route
                for k in range(1, len(rou)):
                    rou_test = rou[:k] + [cust] + rou[k:]

                    if check_violation(rou_test, 5)[0]:
                        new_type2 = route_type(rou_test)
                        adjust_cost2 = check_violation(rou_test, new_type2)[1]
                        cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new = rou_test
                            move_to_route = j


        if best_cut_cost > 1e-5:
            # print('shift1 good', best_cut_cost)
            sol_in1[move_to_route] = route_new
            sol_type1[move_to_route] = route_type(route_new)
            if move_to_route != c_loc[0]:  # moving to a different route
                sol_in1[c_loc[0]] = route_ing
                sol_type1[c_loc[0]] = route_type(route_ing)
        elif sa_lns and best_cut_cost < -1e-5:
            prb = random.uniform(0, 1)
            if np.exp(best_cut_cost/curr_temp) > prb:
                # print('shift1', best_cut_cost)
                sol_in1[move_to_route] = route_new
                sol_type1[move_to_route] = route_type(route_new)
                if move_to_route != c_loc[0]:  # moving to a different route
                    sol_in1[c_loc[0]] = route_ing
                    sol_type1[c_loc[0]] = route_type(route_ing)



        # return sol_in1


    def shift_2_cust(self, sol_in2, cust, c_loc, curr_temp, sol_type2, sa_lns):
        """试着把2个连续的客户搬到他们可以安置的地方，看看他们搬家是否能降低总成本."""

        route_ing = copy.deepcopy(sol_in2[c_loc[0]])
        route_new = route_ing
        move_to_route = c_loc[0]
        orgn_type1 = sol_type2[c_loc[0]]
        cust_folw = route_ing[c_loc[1]+1]
        origin_cost1 = check_violation(route_ing, orgn_type1)[1]
        route_ing.remove(cust)  # remove c in the current route
        del route_ing[c_loc[1]]  # remove customer following c
        new_type1 = route_type(route_ing)
        adjust_cost1 = check_violation(route_ing, new_type1)[1]
        best_cut_cost0 = -1000
        best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
        for j, rou in enumerate(sol_in2):
            orgn_type2 = sol_type2[j]
            origin_cost2 = check_violation(rou, orgn_type2)[1]
            if j == c_loc[0]:  # moving in the same route
                for k in range(1, len(route_ing)):
                    if k == c_loc[1]:
                        continue
                    rou_test = route_ing[:k] + [cust, cust_folw] + route_ing[k:]
                    if check_violation(rou_test, orgn_type2)[0]:
                        adjust_cost2 = check_violation(rou_test, orgn_type2)[1]
                        cost_cut_test = origin_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new = rou_test
                            move_to_route = j


            else:  # moving to a different route
                for k in range(1, len(rou)):
                    rou_test = rou[:k] + [cust, cust_folw] + rou[k:]
                    if check_violation(rou_test, 5)[0]:
                        new_type2 = route_type(rou_test)
                        adjust_cost2 = check_violation(rou_test, new_type2)[1]
                        cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new = rou_test
                            move_to_route = j


        if best_cut_cost > 1e-5:
            # print('shift2 good', best_cut_cost)
            sol_in2[move_to_route] = route_new
            sol_type2[move_to_route] = route_type(route_new)
            if move_to_route != c_loc[0]:  # moving to a different route
                sol_in2[c_loc[0]] = route_ing
                sol_type2[c_loc[0]] = route_type(route_ing)

        elif sa_lns and best_cut_cost < -1e-5:
            prb = random.uniform(0, 1)
            if np.exp(best_cut_cost / curr_temp) > prb:
                # print('shift2', best_cut_cost)
                sol_in2[move_to_route] = route_new
                sol_type2[move_to_route] = route_type(route_new)
                if move_to_route != c_loc[0]:  # moving to a different route
                    sol_in2[c_loc[0]] = route_ing
                    sol_type2[c_loc[0]] = route_type(route_ing)

        # return sol_in2



    def shift_3_cust(self, sol_in6, cust, c_loc, curr_temp, sol_type6, sa_lns):
        """试着把3个连续的客户搬到他们可以安置的地方，看看他们搬家是否能降低总成本."""

        route_ing = copy.deepcopy(sol_in6[c_loc[0]])
        route_new = route_ing
        move_to_route = c_loc[0]
        orgn_type1 = sol_type6[c_loc[0]]
        cust_folw1 = route_ing[c_loc[1] + 1]
        cust_folw2 = route_ing[c_loc[1] + 2]
        origin_cost1 = check_violation(route_ing, orgn_type1)[1]
        route_ing.remove(cust)  # remove c in the current route
        del route_ing[c_loc[1]]  # remove customer following c
        del route_ing[c_loc[1]]  # remove customer following following c
        new_type1 = route_type(route_ing)
        adjust_cost1 = check_violation(route_ing, new_type1)[1]
        best_cut_cost0 = -1000
        best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
        for j, rou in enumerate(sol_in6):
            orgn_type2 = sol_type6[j]
            origin_cost2 = check_violation(rou, orgn_type2)[1]
            if j == c_loc[0]:  # moving in the same route
                for k in range(1, len(route_ing)):
                    if k == c_loc[1]:
                        continue
                    rou_test = route_ing[:k] + [cust, cust_folw1, cust_folw2] + route_ing[k:]
                    if check_violation(rou_test, orgn_type2)[0]:
                        adjust_cost2 = check_violation(rou_test, orgn_type2)[1]
                        cost_cut_test = origin_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new = rou_test
                            move_to_route = j

            else:  # moving to a different route
                for k in range(1, len(rou)):
                    rou_test = rou[:k] + [cust, cust_folw1, cust_folw2] + rou[k:]
                    if check_violation(rou_test, 5)[0]:
                        new_type2 = route_type(rou_test)
                        adjust_cost2 = check_violation(rou_test, new_type2)[1]
                        cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new = rou_test
                            move_to_route = j


        if best_cut_cost > 1e-5:
            # print('shift3 good', best_cut_cost)
            sol_in6[move_to_route] = route_new
            sol_type6[move_to_route] = route_type(route_new)
            if move_to_route != c_loc[0]:  # moving to a different route
                sol_in6[c_loc[0]] = route_ing
                sol_type6[c_loc[0]] = route_type(route_ing)

        elif sa_lns and best_cut_cost < -1e-5:

            prb = random.uniform(0, 1)
            if np.exp(best_cut_cost / curr_temp) > prb:
                # print('shift3', best_cut_cost)
                sol_in6[move_to_route] = route_new
                sol_type6[move_to_route] = route_type(route_new)
                if move_to_route != c_loc[0]:  # moving to a different route
                    sol_in6[c_loc[0]] = route_ing
                    sol_type6[c_loc[0]] = route_type(route_ing)


    def exchange_1_cust(self, sol_in3, cust, c_loc, curr_temp, sol_type3, sa_lns):
        """在可行的情况下交换两个客户的位置(是否同路线)，看看能不能降低总成本."""

        route_ing = copy.deepcopy(sol_in3[c_loc[0]])

        route_new_1 = route_ing
        route_new_2 = route_ing
        exch_to_route = c_loc[0]
        orgn_type1 = sol_type3[exch_to_route]
        origin_cost1 = check_violation(route_ing, orgn_type1)[1]
        # route_ing.remove(cust)  # move c in the current route
        # adjust_cost1 = check_violation(route_ing)[1]
        best_cut_cost0 = -1000
        best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
        for j, rou in enumerate(sol_in3):
            orgn_type2 = sol_type3[j]
            origin_cost2 = check_violation(rou, orgn_type2)[1]
            if j == c_loc[0]:  # exchange in the same route
                for k in range(1, len(rou)-1):
                    if k == c_loc[1]:
                        continue
                    rou_test = copy.deepcopy(sol_in3[c_loc[0]])
                    rou_test[k], rou_test[c_loc[1]] = rou_test[c_loc[1]], rou_test[k]
                    if check_violation(rou_test, orgn_type2)[0]:
                        adjust_cost2 = check_violation(rou_test, orgn_type2)[1]
                        cost_cut_test = origin_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new_1 = rou_test
                            route_new_2 = rou_test
                            exch_to_route = j

            else:  # exchange to a different route
                for k in range(1, len(rou)-1):
                    rou_test_1 = copy.deepcopy(sol_in3[c_loc[0]])
                    rou_test_2 = copy.deepcopy(rou)
                    rou_test_1[c_loc[1]] = rou[k]
                    rou_test_2[k] = cust
                    if check_violation(rou_test_1, 5)[0] and check_violation(rou_test_2, 5)[0]:
                        new_type1 = route_type(rou_test_1)
                        new_type2 = route_type(rou_test_2)
                        adjust_cost1 = check_violation(rou_test_1, new_type1)[1]
                        adjust_cost2 = check_violation(rou_test_2, new_type2)[1]
                        cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new_1 = rou_test_1
                            route_new_2 = rou_test_2
                            exch_to_route = j



        if best_cut_cost > 1e-5:
            # print('exchange1 good', best_cut_cost)
            sol_in3[c_loc[0]] = route_new_1
            sol_in3[exch_to_route] = route_new_2
            sol_type3[c_loc[0]] = route_type(route_new_1)
            sol_type3[exch_to_route] = route_type(route_new_2)

        elif sa_lns and best_cut_cost < -1e-5:
            prb = random.uniform(0, 1)
            if np.exp(best_cut_cost / curr_temp) > prb:
                # print('exchange1', best_cut_cost)
                sol_in3[c_loc[0]] = route_new_1
                sol_in3[exch_to_route] = route_new_2
                sol_type3[c_loc[0]] = route_type(route_new_1)
                sol_type3[exch_to_route] = route_type(route_new_2)

        # return sol_in3


    def exchange_2_cust(self, sol_in4, cust, c_loc, curr_temp, sol_type4, sa_lns):
        """将连续2个客户的位置交换到另外2个客户的位置，看看是否可以降低成本."""

        route_ing = copy.deepcopy(sol_in4[c_loc[0]])
        route_new_1 = route_ing
        route_new_2 = route_ing
        cust_folw = route_ing[c_loc[1] + 1]
        exch_to_route = c_loc[0]
        origin_cost1 = check_violation(route_ing, sol_type4[c_loc[0]])[1]
        # route_ing.remove(cust)           # move c in the current route
        # adjust_cost1 = check_violation(route_ing)[1]
        best_cut_cost0 = -1000
        best_cut_cost = best_cut_cost0  # best cost cut of moving this customer  最好的成本削减搬迁该客户
        for j, rou in enumerate(sol_in4):
            origin_cost2 = check_violation(rou, sol_type4[j])[1]
            if j != c_loc[0] and len(rou) >= 4:  # 交换到不同的路线
                for k in range(1, len(rou) - 2):
                    rou_test_1 = copy.deepcopy(sol_in4[c_loc[0]])
                    rou_test_2 = copy.deepcopy(rou)
                    rou_test_1[c_loc[1]], rou_test_1[c_loc[1] + 1] = rou[k], rou[k + 1]
                    rou_test_2[k], rou_test_2[k + 1] = cust, cust_folw
                    if check_violation(rou_test_1, 5)[0] and check_violation(rou_test_2, 5)[0]:
                        new_type1 = route_type(rou_test_1)
                        new_type2 = route_type(rou_test_2)
                        adjust_cost1 = check_violation(rou_test_1, new_type1)[1]
                        adjust_cost2 = check_violation(rou_test_2, new_type2)[1]
                        cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new_1 = rou_test_1
                            route_new_2 = rou_test_2
                            exch_to_route = j



        if best_cut_cost > 1e-5:
            # print('exchange2 good', best_cut_cost)
            sol_in4[c_loc[0]] = route_new_1
            sol_in4[exch_to_route] = route_new_2
            sol_type4[c_loc[0]] = route_type(route_new_1)
            sol_type4[exch_to_route] = route_type(route_new_2)

        elif sa_lns and best_cut_cost < -1e-5:
            prb = random.uniform(0, 1)
            if np.exp(best_cut_cost / curr_temp) > prb:
                # print('exchange2', best_cut_cost)
                sol_in4[c_loc[0]] = route_new_1
                sol_in4[exch_to_route] = route_new_2
                sol_type4[c_loc[0]] = route_type(route_new_1)
                sol_type4[exch_to_route] = route_type(route_new_2)

        # return sol_in4


    def two_exchange_sol(self, sol_in5, curr_temp, sol_type5, sa_lns):
        """Two-Exchange operator: For two customers i and j on the same route where i is visited before j,
        remove arcs (i,i+),(j,j+); add arcs (i,j),(i+,j+); and reverse the orientation of the arcs between i+ and j.
        Given a solution, check all possible neighborhood.
        """
        solu = copy.deepcopy(sol_in5)
        best_cut_cost0 = -1000
        best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
        adjust_rou_ind = 0
        route_new = sol_in5[adjust_rou_ind]

        for i, rou in enumerate(solu):
            if len(rou) >= 6:
                orgn_type = sol_type5[i]
                origin_cost = check_violation(rou, orgn_type)[1]
                for k in range(1, len(rou)-4):
                    for l in range(k+3, len(rou)-1):
                        route_test = copy.deepcopy(rou)
                        route_test[k], route_test[l] = route_test[l], route_test[k]
                        route_test[k+1: l] = route_test[l-1:k:-1]  # middle reverse
                        if check_violation(route_test, orgn_type)[0]:
                            adjust_cost = check_violation(route_test, orgn_type)[1]
                            if origin_cost - adjust_cost > best_cut_cost:
                                best_cut_cost = origin_cost - adjust_cost
                                adjust_rou_ind = i
                                route_new = route_test



        if best_cut_cost > 1e-5:
            # print('2exchange good', best_cut_cost)
            sol_in5[adjust_rou_ind] = route_new
            sol_type5[adjust_rou_ind] = route_type(route_new)

        elif sa_lns and best_cut_cost < -1e-5:
            prb = random.uniform(0, 1)
            if np.exp(best_cut_cost / curr_temp) > prb:
                # print('2exchange', best_cut_cost)
                sol_in5[adjust_rou_ind] = route_new
                sol_type5[adjust_rou_ind] = route_type(route_new)

        # return sol_in5


    def two_opt(self, sol_in7, cust, c_loc, curr_temp, sol_type7, sa_lns):
        """2-opt*: given customer i in route a and customer j in route b, exchange the following sequences of i and j.
        for example, initial route a: ...-i-i1-i2-..., initial route b: ...-j-j1-j2-...
        New route a: ...-i-j1-j2-..., new route b: ...-j-i1-i2-..."""

        route_ing = copy.deepcopy(sol_in7[c_loc[0]])

        route_new_1 = route_ing
        route_new_2 = route_ing
        exch_to_route = c_loc[0]
        orgn_type1 = sol_type7[c_loc[0]]
        origin_cost1 = check_violation(route_ing, orgn_type1)[1]
        # route_ing.remove(cust)  # move c in the current route
        # adjust_cost1 = check_violation(route_ing)[1]
        best_cut_cost0 = -1000
        best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
        for j, rou in enumerate(sol_in7):
            orgn_type2 = sol_type7[j]
            origin_cost2 = check_violation(rou, orgn_type2)[1]
            if j != c_loc[0]:  # 2-opt* operator has to be implemented in 2 different routes
                for k in range(1, len(rou) - 1):
                    rou_test_1 = sol_in7[c_loc[0]][:c_loc[1]] + rou[k:]
                    rou_test_2 = rou[:k] + sol_in7[c_loc[0]][c_loc[1]:]
                    if check_violation(rou_test_1, 5)[0] and check_violation(rou_test_2, 5)[0]:
                        new_type1 = route_type(rou_test_1)
                        new_type2 = route_type(rou_test_2)
                        adjust_cost1 = check_violation(rou_test_1, new_type1)[1]
                        adjust_cost2 = check_violation(rou_test_2, new_type2)[1]
                        cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                        if cost_cut_test > best_cut_cost:
                            best_cut_cost = cost_cut_test
                            route_new_1 = rou_test_1
                            route_new_2 = rou_test_2
                            exch_to_route = j


        if best_cut_cost > 1e-5:
            # print('2opt* good', best_cut_cost)
            sol_in7[c_loc[0]] = route_new_1
            sol_in7[exch_to_route] = route_new_2
            sol_type7[c_loc[0]] = route_type(route_new_1)
            sol_type7[exch_to_route] = route_type(route_new_2)

        elif sa_lns and best_cut_cost < -1e-5:
            prb = random.uniform(0, 1)
            if np.exp(best_cut_cost / curr_temp) > prb:
                # print('2opt*', best_cut_cost)
                sol_in7[c_loc[0]] = route_new_1
                sol_in7[exch_to_route] = route_new_2
                sol_type7[c_loc[0]] = route_type(route_new_1)
                sol_type7[exch_to_route] = route_type(route_new_2)


    def lns_sa(self, sol_in, veh_type_in, cost_in):
        """基于7个操作符的邻域搜索。在每次迭代中，随机选择一个操作符."""

        itr_cost = []
        solu = copy.deepcopy(sol_in)
        solu_type = copy.deepcopy(veh_type_in)
        best_solu = sol_in
        best_val = cost_in
        tabu_list = []
        random.seed(10)
        itr = 0
        temp = initial_temp
        t_run = time.time()
        while temp > stop_temp:
            itr += 1
            print(itr)
            if itr <= 0:
                sa_lns = True  # use sa or lns
            else:
                sa_lns = False
            c = random.randint(1, store_num - 1)  #   随机生成移动客户
            while c in tabu_list:
                c = random.randint(1, store_num - 1)  # randint(a, b), both a and b are selectable
            c_loc = cust_loc(solu, c)

            if len(solu[c_loc[0]]) < 4: 
        #  客户号码小于2，只能执行shift1和exchange1操作
                wheel_value1 = random.uniform(0, 1)
                if wheel_value1 < 0.45:
                    self.shift_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
                elif wheel_value1 < 0.9:
                    self.exchange_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
                else:
                    self.two_opt(solu, c, c_loc, temp, solu_type, sa_lns)

            #   客户数量2个以上，可实现所有操作
            elif len(solu[c_loc[0]]) >= 4 and c_loc[1] <= len(solu[c_loc[0]]) - 3:
                wheel_value2 = random.uniform(0, 1)
                if wheel_value2 < 0.2:
                    self.shift_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
                elif wheel_value2 < 0.4:
                    self.shift_2_cust(solu, c, c_loc, temp, solu_type, sa_lns)
                elif wheel_value2 < 0.6:
                    self.exchange_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
                elif wheel_value2 < 0.8:
                    self.exchange_2_cust(solu, c, c_loc, temp, solu_type, sa_lns)
                else:
                    self.two_opt(solu, c, c_loc, temp, solu_type, sa_lns)


            if itr % 100 == 0:  
                #   每200次迭代实现两次交换操作符
                self.two_exchange_sol(solu, temp, solu_type, sa_lns)


            temp -= delta
            tabu_list.append(c)
            if len(tabu_list) > 100:
                tabu_list.pop(0)


            cost_i = of.print_result(solu, solu_type, False)
            # print(solu_type)
            itr_cost.append(cost_i)
            if cost_i < best_val:
                best_solu = solu
                best_val = cost_i


            t_run = time.time()

        # Adjust0: delete [0, 0] routes
        adjust_sol0 = []
        for route0 in best_solu:
            if len(route0) <= 2:  # [0, 0] route
                continue
            else:
                adjust_sol0.append(route0)

        #   调整1:如果可能的话，使用小型车辆
        adjust_type = []
        for route1 in adjust_sol0:
            adjust_type.append(route_type(route1))



        return adjust_sol0, adjust_type, best_val, itr_cost   

In [13]:
if __name__ == '__main__':
    t0 = time.time()

    num_id, id_num, loc, num_demd, num_timez = read_data()
    dist_mat, time_mat = earth_dist(loc)
    dist_mat = dist_mat * math.sqrt(2)  
    #   旅行距离和欧氏距离的传递系数为sqrt(2)
    time_mat = time_mat * math.sqrt(2)
    store_num = len(num_id)  # 包括仓库在内的商店数量

    gi = GetInitial()
    of = OutputFormat()

    solu, solu_type, route_way_time = gi.greedy_initial()
    solu_cost = of.print_result(solu, solu_type, False)
    print('Initial number of vehicles: ', len(solu))
    print('Initial solution cost: ', solu_cost)
    # print(solu_veh_type)


    # 模拟退火法

    sa = SimulatedAnnealing()
    initial_temp, stop_temp, delta = 40., 10., 30. / 10000
    new_solu, new_solu_type, new_cost, cost_t = sa.lns_sa(solu, solu_type, solu_cost)
    print('Optimized number of vehicles: ', len(new_solu))
    print('Optimized solution cost: ', new_cost)
    plt.plot(cost_t)
    plt.show()

    # total_dist1 = of.print_route_summary(solution=new_solu, vehicle_type=new_solu_type, if_write=True)
    # of.print_route_detail(solution=new_solu, vehicle_type=new_solu_type, if_write=True)
    # print('Total traveling diatance: ', total_dist1)

    # plot_route(new_solu)

    t1 = time.time()
    print('Total elapsed time: ', t1-t0)  

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 'U' mode is deprecated
  if __name__ == '__main__':


IndexError: index 10001 is out of bounds for axis 0 with size 664

# VRPTW_SA_6operators

In [36]:
def read_data(file_num):

    # 我们发现送货客户必须在12:00之前送达，接货客户必须在13:00之后送达
    cust_need = {0: [1, 0, 0, 480, 1440]}  # {cust_id: [cust_type, weight, volume, first_receive, last_receive], []}
    time_mat = np.zeros([cust_num + charge_num, cust_num + charge_num])
    dist_mat = np.zeros([cust_num + charge_num, cust_num + charge_num])


    with open(r'G:\CDO\海上风电场\LNS\JD-GOC-VRP-master\data\inputnode_1_1601.csv' % file_num, 'rU') as fc:
        reader = csv.reader(fc)
        next(reader)
        next(reader)
        for v in reader:
            # if v[1] == '1':
            #     cust_need[0] = [1, 0, 480, 1440]
            if v[1] == '4':
                cust_need[int(v[0]) - file_code * 10000] = [4, 0, 0, 480, 1440]
            else:
                first_t = v[6].split(':')
                last_t = v[7].split(':')
                cust_need[int(v[0]) - file_code*10000] = [int(v[1]), float(v[4]), float(v[5]), int(first_t[0])*60 + int(first_t[1]), int(last_t[0])*60 + int(last_t[1])]

    # print cust_need  现金需要
    # print cust_time  固化时间

    with open(r'G:\CDO\海上风电场\LNS\JD-GOC-VRP-master\data\inputdistancetime_1_1601\inputdistancetime_1_1601.txt' % file_num, 'r') as fd:
        next(fd)
        for row in fd:
            to_list = row.strip('\n').split(',')
            if int(to_list[1]) > 0:
                from_id = int(to_list[1]) - file_code*10000
            else:
                from_id = int(to_list[1])
            if int(to_list[2]) > 0:
                to_id = int(to_list[2]) - file_code*10000
            else:
                to_id = int(to_list[2])
            dist_mat[from_id, to_id] = int(to_list[3])
            time_mat[from_id, to_id] = int(to_list[4])

    return cust_need, time_mat, dist_mat


In [16]:
def time_nn(last_cust, curr_cust, remain_list, used_resource, rout_len, vehicle_type):
    """给定车辆及其当前来访的客户，返回下一个来访的客户.
    这里我们使用Solomon(1987)提出的面向时间的最近邻启发式。
    The closeness between customer i and j: C_ij = alp1*t_ij + alp2*h_ij + alp3*v_ij,
    t_ij: travel time between i and j (distance);
    h_ij = b_j - (b_i + s_i), b is start service time, and s is service time (idle time);
    v_ij = l_j - (b_i + s_i + t_ij), l is the latest admissible service time, t_ij is the travel time (urgency)
    The low value of C_ij, the better.
    """
    if vehicle_type == 1:
        veh_cap = iveco_cap
    else:
        veh_cap = truck_cap

    visit_cust = [-1, 100000, 600000, 1000]  # [cust_id, next_start, distance, closeness]
    if rout_len > 6:
        pass
    elif cust_need[curr_cust][0] <= 2 or (cust_need[curr_cust][0] == 4 and cust_need[last_cust][0] <= 2):  
        # 当前客户是一个交付
        for cust in remain_list:
            near_charg_id = cust_charge[cust]
            # print 'checking customer: ', cust
            if cust_need[cust][0] == 2:  # 下一个客户是送货
                if used_resource[2] + dist_mat[curr_cust, cust] + dist_mat[cust, near_charg_id] > veh_cap[3]:
                    # print 'd out of battery 1'
                    continue  # 在到达来访客户最近的充电站之前用完电池
                elif dist_mat[curr_cust, cust] > veh_cap[3] - used_resource[2]:
                    # print 'd out of battery 2'
                    continue  # 在拜访客户之前用完电池
                elif used_resource[0] + cust_need[cust][1] > veh_cap[1]:
                    # print 'd weight overload'
                    continue  # weight overload  重量超载
                elif used_resource[1] + cust_need[cust][2] > veh_cap[2]:
                    # print 'd volume overload'
                    continue  # volume overload
                elif used_resource[3] + time_mat[curr_cust, cust] > cust_need[cust][4]:
                    # print 'd last receive time'
                    continue  # 不能在最后一次接收时间之前到达

                else:
                    wait_time = cust_need[cust][3] - (used_resource[3] + time_mat[curr_cust, cust])
                    if wait_time < 0:
                        next_start = used_resource[3] + time_mat[curr_cust, cust]
                        h_ij = time_mat[curr_cust, cust]
                    else:
                        next_start = cust_need[cust][3]
                        h_ij = next_start - used_resource[3]
                    v_ij = cust_need[cust][4] - (used_resource[3] + time_mat[curr_cust, cust])
                    close_ij = alp * time_mat[curr_cust, cust] + bet * h_ij +gam * v_ij  # closeness between i and j
                    if close_ij < visit_cust[3]:
                        visit_cust[0] = cust
                        visit_cust[1] = next_start
                        visit_cust[2] = dist_mat[curr_cust, cust]
                        visit_cust[3] = close_ij
                    else:
                        continue

            else:  # 下一个客户是皮卡车
                if used_resource[2] + dist_mat[curr_cust, cust] + dist_mat[cust, near_charg_id] > veh_cap[3]:
                    # print 'p out of battery 1'
                    continue  #  在到达来访客户最近的充电站之前用完电池
                elif dist_mat[curr_cust, cust] > veh_cap[3] - used_resource[2]:
                    # print 'p out of battery 2'
                    continue  # 在拜访客户之前用完电池
                elif used_resource[3] + time_mat[curr_cust, cust] > cust_need[cust][4]:
                    # print 'p last receive time'
                    continue  # 不能在最后一次接收时间之前到达

                else:
                    wait_time = cust_need[cust][3] - (used_resource[3] + time_mat[curr_cust, cust])
                    if wait_time < 0:
                        next_start = used_resource[3] + time_mat[curr_cust, cust]
                        h_ij = time_mat[curr_cust, cust]
                    else:
                        next_start = cust_need[cust][3]
                        h_ij = next_start - used_resource[3]
                    v_ij = cust_need[cust][4] - (used_resource[3] + time_mat[curr_cust, cust])
                    close_ij = alp * time_mat[curr_cust, cust] + bet * h_ij +gam * v_ij  # i和j之间的紧密关系
                    if close_ij < visit_cust[3]:
                        visit_cust[0] = cust
                        visit_cust[1] = next_start
                        visit_cust[2] = dist_mat[curr_cust, cust]
                        visit_cust[3] = close_ij
                    else:
                        continue


    else:  # 目前的客户是皮卡
        for cust in remain_list:
            near_charg_id = cust_charge[cust]
            if cust_need[cust][0] == 2:
                continue  # 不再交付客户
            elif used_resource[2] + dist_mat[curr_cust, cust] + dist_mat[cust, near_charg_id] > veh_cap[3]:
                continue  # 在到达来访客户最近的充电站之前用完电池
            elif dist_mat[curr_cust, cust] > veh_cap[3] - used_resource[2]:
                continue  # run out of battery before getting to the visiting customer
            elif used_resource[0] + cust_need[cust][1] > veh_cap[1]:
                continue  # weight overload
            elif used_resource[1] + cust_need[cust][2] > veh_cap[2]:
                continue  # volume overload
            elif used_resource[3] + time_mat[curr_cust, cust] > cust_need[cust][4]:
                continue  # can not arrive before last receive time

            else:
                wait_time = cust_need[cust][3] - (used_resource[3] + time_mat[curr_cust, cust])
                if wait_time < 0:
                    next_start = used_resource[3] + time_mat[curr_cust, cust]
                    h_ij = time_mat[curr_cust, cust]
                else:
                    next_start = cust_need[cust][3]
                    h_ij = next_start - used_resource[3]
                v_ij = cust_need[cust][4] - (used_resource[3] + time_mat[curr_cust, cust])
                close_ij = alp * time_mat[curr_cust, cust] + bet * h_ij + gam * v_ij  # closeness between i and j
                if close_ij < visit_cust[3]:
                    visit_cust[0] = cust
                    visit_cust[1] = next_start
                    visit_cust[2] = dist_mat[curr_cust, cust]
                    visit_cust[3] = close_ij
                else:
                    continue

    # if visit_cust[0] == -1:  # no customer to visit
    #     if dist_mat[curr_cust, 0] <= truck_cap[3] - used_resource[2]:  # can get back to depot
    #         visit_cust[0] = 0
    #         visit_cust[1] = used_resource[-1] + time_mat[curr_cust, 0]
    #     else:
    #         visit_cust[0] = cust_charge[curr_cust]  # get to the nearest charge station
    #         visit_cust[1] = used_resource[-1] + time_mat[curr_cust, visit_cust[0]]

    if visit_cust[0] == -1:  # no customer to visit
        if 2 <= cust_need[curr_cust][0] <= 3:  # for customers, first choose to get charged if no customers to visit
            visit_cust[0] = cust_charge[curr_cust]  # get to the nearest charge station
            visit_cust[1] = used_resource[-1] + time_mat[curr_cust, visit_cust[0]]
        else:  # 对于充电站和车场，如果没有客户来访，返回车场
            visit_cust[0] = 0
            visit_cust[1] = used_resource[-1] + time_mat[curr_cust, 0]

    return visit_cust 

In [19]:
def check_violation(route, vehicle_type):
    """使用大型车辆(卡车)检查路线是否可行，并返回检查结果和路线成本."""
    if len(route) == 2:  # [0, 0] route
        return True, 0
    elif len(route) == 3 and cust_need[route[1]][0] == 4:  # [0, charge, 0] route
        return True, 0
    else:
        # 1 .离开的时间，2 .累计的距离，3 .累计的鸣声，4 .累计的音量
        # 4 .到达顾客处时，累积重量，5 .累积体积
        accu_res = [480, 0, 0, 0, 0, 0]
        if vehicle_type == 1:
            veh_cap = iveco_cap
        elif vehicle_type == 2:
            veh_cap = truck_cap
        else:
            print ('Input wrong vehicle type!', vehicle_type)

        fixed_cost = veh_cap[5]
        trans_cost = 0
        wait_cost = 0
        charge_cost = 0
        if time_mat[0, route[1]] + 480 < cust_need[route[1]][3]:
            accu_res[0] = cust_need[route[1]][3] - time_mat[0, route[1]]
        for i in range(len(route) - 1):
            last_cust = route[i]
            curr_cust = route[i+1]

            # 检查离开时间
            arr_time = accu_res[0] + time_mat[last_cust, curr_cust]
            if arr_time < cust_need[curr_cust][3]:
                accu_res[0] = cust_need[curr_cust][3] + oper_t
                wait_time = cust_need[curr_cust][3] - arr_time
                wait_cost += (wait_time / 60. * wait_cost0)
            elif arr_time <= cust_need[curr_cust][4]:
                accu_res[0] = arr_time + oper_t
            else:
                # print 'Infeasible route!(Service Time Error.)'
                return False, 1000000

            # 检查车辆最大距离
            trans_cost += (dist_mat[last_cust, curr_cust] * veh_cap[4])

            if 2 <= cust_need[last_cust][0] <= 3:
                accu_res[1] += dist_mat[last_cust, curr_cust]
            else:
                accu_res[1] = dist_mat[last_cust, curr_cust]
            if accu_res[1] > veh_cap[3]:
                # print 'Infeasible route!(Max Distance Error.)'
                return False, 1000000

            # 检查车辆最大重量和体积
            if cust_need[curr_cust][0] == 1:
                accu_res[2:] = [0, 0, 0, 0]
            elif cust_need[curr_cust][0] == 2:
                accu_res[2] += cust_need[curr_cust][1]
                accu_res[3] += cust_need[curr_cust][2]
            elif cust_need[curr_cust][0] == 3:
                accu_res[4] += cust_need[curr_cust][1]
                accu_res[5] += cust_need[curr_cust][2]
            else:
                pass

            if accu_res[2] > veh_cap[1] or accu_res[3] > veh_cap[2] or accu_res[4] > veh_cap[1] or accu_res[5] > veh_cap[2]:
                # print 'Infeasible route!(Max Weight/Volume Error.)'
                return False, 1000000

            if cust_need[last_cust][0] == 4:
                charge_cost += charg_cost0

    # print trans_cost, wait_cost, charge_cost, fixed_cost
    return True, trans_cost + wait_cost + charge_cost + fixed_cost


In [20]:
def lns_initial(small_veh):
    """
    这是一个大型的VRPTW邻居搜索算法,
    我们根据等待时间最短来选择下一个来访客户.
    """
    sol = []  # [[0;2;5;0;4;6;0],[],...]
    visited_p = []
    to_vist = [i+1 for i in range(cust_num-1)]  # [1,5,8,...]
    itr = 0

    while len(to_vist) > 0:
        itr += 1
        if itr < small_veh:
            vehicle_type0 = 1
        else:
            vehicle_type0 = 2
        used_res = [0, 0, 0, 480]  # used weight, volume and distance of the vehicle, leave time
        veh_rout = [0]

        # print '\nA new vehicle will be used.'
        while True:
            curr_cust = veh_rout[-1]
            if len(veh_rout) == 1:
                last_cust = 0
            else:
                last_cust = veh_rout[-2]
            # print used_res
            next_one = time_nn(last_cust, curr_cust, to_vist, used_res, len(veh_rout), vehicle_type0)
            next_cust, next_start = next_one[0], next_one[1]

            if next_cust == 0:  # next visiting customer is depot
                if curr_cust == 0:
                    # print 'The current vehicle ends.'
                    break
                else:
                    used_res[:3] = [0, 0, 0]
                    used_res[3] += (time_mat[curr_cust, next_cust] + depot_t)
                    # print 'Get back to the depot, and ready for a new round.'

            elif cust_need[next_cust][0] == 2:  # next visiting customer is delivery customer
                used_res[0] += cust_need[next_cust][1]
                used_res[1] += cust_need[next_cust][2]
                used_res[2] += dist_mat[curr_cust, next_cust]
                used_res[3] = (next_start + oper_t)

            elif cust_need[next_cust][0] == 3:  # next visiting customer is pickup customer
                if curr_cust == 0:  # current customer is depot
                    used_res[0] = cust_need[next_cust][1]
                    used_res[1] = cust_need[next_cust][2]
                    used_res[2] = dist_mat[curr_cust, next_cust]
                    used_res[3] = (next_start + oper_t)
                elif cust_need[curr_cust][0] <= 2:  # current customer is delivery customer
                    used_res[0] = cust_need[next_cust][1]
                    used_res[1] = cust_need[next_cust][2]
                    used_res[2] += dist_mat[curr_cust, next_cust]
                    used_res[3] = (next_start + oper_t)
                else:  # current customer is pickup customer or charge station
                    used_res[0] += cust_need[next_cust][1]
                    used_res[1] += cust_need[next_cust][2]
                    used_res[2] += dist_mat[curr_cust, next_cust]
                    used_res[3] = (next_start + oper_t)

            else:  # next visiting customer is a charge station
                used_res[2] = 0
                used_res[3] += (time_mat[curr_cust, next_cust] + charg_t)

            veh_rout.append(next_cust)
            # print 'Vehicle used resource: ', used_res
            if cust_need[next_cust][0] == 2 or cust_need[next_cust][0] == 3:
                # print 'visited customer', next_cust
                to_vist.remove(next_cust)
                if cust_need[next_cust][0] == 3:
                    visited_p.append(next_cust)


        if cust_need[veh_rout[-2]][0] == 4:  # 最后访问一个充电站，判断是否需要这个充电站
            veh_rout_test = copy.deepcopy(veh_rout)
            veh_rout_test.pop(-2)
            if check_violation(veh_rout_test, 1)[0]:
                if check_violation(veh_rout_test, 1)[1] < check_violation(veh_rout, 1)[1]:
                    sol.append(veh_rout_test)
                    continue
            elif check_violation(veh_rout_test, 2)[0]:
                if check_violation(veh_rout_test, 2)[1] < check_violation(veh_rout, 1)[1]:
                    sol.append(veh_rout_test)
                    continue
        sol.append(veh_rout)
        # print 'Last point 0 earliest leave time: ', int(used_res[-1]) / 60, ':', int(used_res[-1]) % 60
        # print 'Route %s is: ' % itr, veh_rout
        # print '*'*10, 'Iteration:', itr, '*'*10



    return sol


In [21]:
def get_cost(solution, veh_type, if_write, run_t=289.3):
    """给定列表中保存的解决方案，计算解决方案的总成本.
    将解决方案以所需格式写入本地."""

    result = [['trans_code', 'vehicle_type', 'dist_seq', 'distribute_lea_tm', 'distribute_arr_tm', 'distance', 'trans_cost', 'charge_cost', 'wait_cost', 'fixed_use_cost', 'total_cost', 'charge_cnt']]
    total_cost = 0
    veh_code = 0
    for k, veh in enumerate(solution):
        if veh_type[k] == 1:
            trans0 = iveco_cap[4]
            fix0 = iveco_cap[5]
        else:
            trans0 = truck_cap[4]
            fix0 = truck_cap[5]

        # get the output format
        route = [0] * len(result[0])
        veh_code += 1
        route[0] = 'DP' + str(veh_code).zfill(4)  # vehicle name
        route[1] = veh_type[k]  # vehicle type
        route_ele = []
        for ele in veh:
            if ele == 0:
                route_ele.append(str(ele))
            else:
                route_ele.append(str(ele + file_code * 10000))
        route[2] = ';'.join(route_ele)  # route


        total_cost += fix0
        total_cost += dist_mat[0, veh[1]] * trans0
        if time_mat[0, veh[1]] + start_t <= cust_need[veh[1]][3]:
            t = cust_need[veh[1]][3] + oper_t
            time_out = int(cust_need[veh[1]][3] - time_mat[0, veh[1]])
            route[3] = str(time_out / 60) + ':' + str(time_out % 60).zfill(2)  # vehicle out time
        else:

            t = time_mat[0, veh[1]] + start_t + oper_t
            route[3] = str(start_t / 60) + ':' + str(start_t % 60).zfill(2)  # vehicle out time
        total_wait_cost = 0
        for i in range(2, len(veh)-1):  # can not wait at the first 2 points
            total_cost += (dist_mat[veh[i-1], veh[i]] * trans0)
            if cust_need[veh[i]][0] == 4:
                total_cost += charg_cost0
            wait_t = cust_need[veh[i]][3] - (t + time_mat[veh[i-1], veh[i]])
            if wait_t > 0:
                # print veh[i-1], veh[i], wait_t
                total_cost += (wait_t/60. * wait_cost0)
                total_wait_cost += (wait_t/60. * wait_cost0)
                t = cust_need[veh[i]][3] + oper_t
            else:
                if veh[i] == 0:
                    t += (time_mat[veh[i-1], veh[i]] + depot_t)
                else:
                    t += (time_mat[veh[i - 1], veh[i]] + oper_t)
            if veh[i] == 0:  # 返回仓库并再次出发，等待费用为1小时
                total_cost += wait_cost0
                total_wait_cost += wait_cost0
            # print veh[i], str(int(t) / 60) + ':' + str(int(t) % 60).zfill(2)


        in_time = int(t + time_mat[veh[-2], 0])
        route[4] = str(in_time / 60) + ':' + str(in_time % 60).zfill(2)  # vehicle back time
        total_dist = 0
        total_charg_cost = 0
        total_charg_cnt = 0
        for j in range(len(veh) - 1):
            total_dist += dist_mat[veh[j], veh[j + 1]]
            if veh[j] >= cust_num:
                total_charg_cost += charg_cost0
                total_charg_cnt += 1
        route[5] = int(total_dist)  # total distance
        route[6] = round(route[5] * trans0, 2)  # transfer cost
        route[7] = total_charg_cost  # total charge cost
        route[8] = total_wait_cost
        route[9] = fix0  # vehicle fixed cost
        route[10] = route[6] + route[7] + route[8] + route[9]  # total cost
        route[11] = total_charg_cnt  # charge count

        result.append(route)
        # print route
        total_cost += dist_mat[veh[-2], 0] * trans0
        # print 'Last leave time: ', int(t) / 60, ':', int(t) % 60
        # print 'total distances: ', route[5]


    if if_write:
        with open(r'G:\CDO\海上风电场\LNS\JD-GOC-VRP-master\data\inputdistancetime_1_1601\Result_%s_%s.csv' % (file_code, run_t), 'wb') as fw:
            writer = csv.writer(fw)
            for v in result:
                writer.writerow(v)

    return total_cost


In [22]:
def vehicle_type_adjust(solution):
    """给出一个只使用大型卡车的解决方案，看看是否可以用小型汽车来代替，以节省成本."""
    type_list = []
    for veh in solution:
        typ = 2
        wei_shou = [0]  # pickup weight
        wei_song = [0]  # delivery weight
        vol_shou = [0]
        vol_song = [0]
        distance = []  # 各点距离
        for i in range(len(veh) - 1):
            if 2 <= cust_need[veh[i]][0] <= 3:
                distance0 = distance[-1] + dist_mat[veh[i], veh[i+1]]
                distance.append(distance0)
            else:
                distance0 = dist_mat[veh[i], veh[i+1]]
                distance.append(distance0)

            wei_song0, wei_shou0, vol_song0, vol_shou0 = 0, 0, 0, 0
            if cust_need[veh[i+1]][0] == 2:
                wei_song0 = wei_song[-1] + cust_need[veh[i+1]][1]
                vol_song0 = vol_song[-1] + cust_need[veh[i+1]][2]
            elif cust_need[veh[i+1]][0] == 3:
                wei_shou0 = wei_shou[-1] + cust_need[veh[i + 1]][1]
                vol_shou0 = vol_shou[-1] + cust_need[veh[i + 1]][2]
            elif veh[i+1] == 0:  # 返回到仓库初始化车辆资源
                wei_song0, wei_shou0, vol_song0, vol_shou0 = 0, 0, 0, 0
            else:
                continue
            wei_song.append(wei_song0)
            wei_shou.append(wei_shou0)
            vol_song.append(vol_song0)
            vol_shou.append(vol_shou0)

        if max(wei_song) > 2.5 or max(wei_shou) > 2.5 or max(vol_song) > 16 or max(vol_shou) > 16 or max(distance) > 120000:
            print ('Shit!!!')
            print ('Error route: ', veh)
            print ('wei_song wei_shou vol_song vol_shou distance: ', max(wei_song), max(wei_shou), max(vol_song), max(vol_shou), max(distance))
        if max(wei_song) <= iveco_cap[1] and max(wei_shou) <= iveco_cap[1] and max(vol_song) <= iveco_cap[2] and max(vol_shou) <= iveco_cap[2] and max(distance) <= iveco_cap[3]:
            typ = 1

        type_list.append(typ)

    return type_list 

In [23]:
def route_type(route):
    """给定一条路线，返回该路线的车辆类型。小型车辆第一，大型车辆第二."""
    typ = 2
    wei_shou = [0]  # pickup weight
    wei_song = [0]  # delivery weight
    vol_shou = [0]
    vol_song = [0]
    distance = []  # 各点距离
    for i in range(len(route) - 1):
        if 2 <= cust_need[route[i]][0] <= 3:
            distance0 = distance[-1] + dist_mat[route[i], route[i + 1]]
            distance.append(distance0)
        else:
            distance0 = dist_mat[route[i], route[i + 1]]
            distance.append(distance0)

        wei_song0, wei_shou0, vol_song0, vol_shou0 = 0, 0, 0, 0
        if cust_need[route[i + 1]][0] == 2:
            wei_song0 = wei_song[-1] + cust_need[route[i + 1]][1]
            vol_song0 = vol_song[-1] + cust_need[route[i + 1]][2]
        elif cust_need[route[i + 1]][0] == 3:
            wei_shou0 = wei_shou[-1] + cust_need[route[i + 1]][1]
            vol_shou0 = vol_shou[-1] + cust_need[route[i + 1]][2]
        elif route[i + 1] == 0:                            # 返回到仓库初始化车辆资源
            wei_song0, wei_shou0, vol_song0, vol_shou0 = 0, 0, 0, 0
        else:
            continue
        wei_song.append(wei_song0)
        wei_shou.append(wei_shou0)
        vol_song.append(vol_song0)
        vol_shou.append(vol_shou0)

    if max(wei_song) > 2.5 or max(wei_shou) > 2.5 or max(vol_song) > 16 or max(vol_shou) > 16 or max(distance) > 120000:
        print ('Shit!!!')
        print ('Error route: ', route)
        print ('wei_song wei_shou vol_song vol_shou distance: ', max(wei_song), max(wei_shou), max(vol_song), max(
            vol_shou), max(distance))
    if max(wei_song) <= iveco_cap[1] and max(wei_shou) <= iveco_cap[1] and max(vol_song) <= iveco_cap[2] and max(
            vol_shou) <= iveco_cap[2] and max(distance) <= iveco_cap[3]:
        typ = 1

    return typ 

In [24]:
def cust_loc(sol, cust):
    """获取一个客户的路线位置和客户位置."""
    cust_ind = []  # [route_loc, cust_loc]
    for i, rt in enumerate(sol):
        if cust in rt:
            cust_ind.append(i)
            cust_ind.append(rt.index(cust))
            return cust_ind

    print ('Costomer not in the solution: ', cust)  

In [25]:
def shift_1_cust(sol_in1, cust, c_loc, curr_temp, sol_type1, sa_lns):
    """试着把一个客户搬到任何可以安置的地方，看看这样做是否能降低总成本."""

    route_ing = copy.deepcopy(sol_in1[c_loc[0]])
    route_new = route_ing
    move_to_route = c_loc[0]
    new_type = 2
    origin_cost1 = check_violation(route_ing, sol_type1[c_loc[0]])[1]
    route_ing.remove(cust)          # 在当前路径上移动c
    adjust_cost1 = min(check_violation(route_ing, 1)[1], check_violation(route_ing, 2)[1])
    best_cut_cost0 = -1000
    best_cut_cost = best_cut_cost0  # 最好的成本削减搬迁该客户
    for j, rou in enumerate(sol_in1):
        origin_cost2 = check_violation(rou, sol_type1[j])[1]
        if j == c_loc[0]:          # 沿着同样的路线移动
            for k in range(1, len(route_ing)):
                if k == c_loc[1]:
                    continue      # 不把它放在原来的位置
                rou_test = route_ing[:k] + [cust] + route_ing[k:]
                if check_violation(rou_test, 1)[0]:
                    adjust_cost2 = check_violation(rou_test, 1)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 1
                elif check_violation(rou_test, 2)[0]:
                    adjust_cost2 = check_violation(rou_test, 2)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 2
        else:                       # 换一条不同的路线
            for k in range(1, len(rou)):
                rou_test = rou[:k] + [cust] + rou[k:]
                if check_violation(rou_test, 1)[0]:
                    adjust_cost2 = check_violation(rou_test, 1)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 1
                elif check_violation(rou_test, 2)[0]:
                    adjust_cost2 = check_violation(rou_test, 2)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 2

    if best_cut_cost > 1e-5:
        print ('shift1 good', best_cut_cost)
        sol_in1[move_to_route] = route_new
        sol_type1[move_to_route] = new_type
        if move_to_route != c_loc[0]:       # 换一条不同的路线
            sol_in1[c_loc[0]] = route_ing
            sol_type1[c_loc[0]] = route_type(route_ing)
    elif sa_lns and best_cut_cost < -1e-5:
        prb = random.uniform(0, 1)
        if np.exp(best_cut_cost/curr_temp) > prb:
            print ('shift1', best_cut_cost)
            sol_in1[move_to_route] = route_new
            sol_type1[move_to_route] = new_type
            if move_to_route != c_loc[0]:    # 换一条不同的路线
                sol_in1[c_loc[0]] = route_ing
                sol_type1[c_loc[0]] = route_type(route_ing)



    # return sol_in1 

In [26]:
def shift_2_cust(sol_in2, cust, c_loc, curr_temp, sol_type2, sa_lns):
    """试着把2个连续的客户搬到他们可以安置的地方，看看他们搬家是否能降低总成本."""

    route_ing = copy.deepcopy(sol_in2[c_loc[0]])
    route_new = route_ing
    move_to_route = c_loc[0]
    new_type = 2
    cust_folw = route_ing[c_loc[1]+1]
    origin_cost1 = check_violation(route_ing, sol_type2[c_loc[0]])[1]
    route_ing.remove(cust)                  # remove c in the current route
    del route_ing[c_loc[1]]                 # remove customer following c
    adjust_cost1 = min(check_violation(route_ing, 1)[1], check_violation(route_ing, 2)[1])
    best_cut_cost0 = -1000
    best_cut_cost = best_cut_cost0          # best cost cut of moving this customer
    for j, rou in enumerate(sol_in2):
        origin_cost2 = check_violation(rou, sol_type2[j])[1]
        if j == c_loc[0]:               # moving in the same route
            for k in range(1, len(route_ing)):
                if k == c_loc[1]:
                    continue
                rou_test = route_ing[:k] + [cust, cust_folw] + route_ing[k:]
                if check_violation(rou_test, 1)[0]:
                    adjust_cost2 = check_violation(rou_test, 1)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 1
                elif check_violation(rou_test, 2)[0]:
                    adjust_cost2 = check_violation(rou_test, 2)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 2
        else:  # moving to a different route
            for k in range(1, len(rou)):
                rou_test = rou[:k] + [cust, cust_folw] + rou[k:]
                if check_violation(rou_test, 1)[0]:
                    adjust_cost2 = check_violation(rou_test, 1)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 1
                elif check_violation(rou_test, 2)[0]:
                    adjust_cost2 = check_violation(rou_test, 2)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 2

    if best_cut_cost > 1e-5:
        print ('shift2 good', best_cut_cost)
        sol_in2[move_to_route] = route_new
        sol_type2[move_to_route] = new_type
        if move_to_route != c_loc[0]:         # moving to a different route
            sol_in2[c_loc[0]] = route_ing
            sol_type2[c_loc[0]] = route_type(route_ing)

    elif sa_lns and best_cut_cost < -1e-5:
        prb = random.uniform(0, 1)
        if np.exp(best_cut_cost / curr_temp) > prb:
            print ('shift2', best_cut_cost)
            sol_in2[move_to_route] = route_new
            sol_type2[move_to_route] = new_type
            if move_to_route != c_loc[0]:     # moving to a different route
                sol_in2[c_loc[0]] = route_ing
                sol_type2[c_loc[0]] = route_type(route_ing)

    # return sol_in2


In [27]:
def shift_3_cust(sol_in6, cust, c_loc, curr_temp, sol_type6, sa_lns):
    """试着把3个连续的客户搬到他们可以安置的地方，看看他们搬家是否能降低总成本."""

    route_ing = copy.deepcopy(sol_in6[c_loc[0]])
    route_new = route_ing
    move_to_route = c_loc[0]
    new_type = 2
    cust_folw1 = route_ing[c_loc[1] + 1]
    cust_folw2 = route_ing[c_loc[1] + 2]
    origin_cost1 = check_violation(route_ing, sol_type6[c_loc[0]])[1]
    route_ing.remove(cust)               # remove c in the current route
    del route_ing[c_loc[1]]                # remove customer following c
    del route_ing[c_loc[1]]              # remove customer following following c
    adjust_cost1 = min(check_violation(route_ing, 1)[1], check_violation(route_ing, 2)[1])
    best_cut_cost0 = -1000
    best_cut_cost = best_cut_cost0        # best cost cut of moving this customer
    for j, rou in enumerate(sol_in6):
        origin_cost2 = check_violation(rou, sol_type6[j])[1]
        if j == c_loc[0]:                 # moving in the same route
            for k in range(1, len(route_ing)):
                if k == c_loc[1]:
                    continue
                rou_test = route_ing[:k] + [cust, cust_folw1, cust_folw2] + route_ing[k:]
                if check_violation(rou_test, 1)[0]:
                    adjust_cost2 = check_violation(rou_test, 1)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 1
                elif check_violation(rou_test, 2)[0]:
                    adjust_cost2 = check_violation(rou_test, 2)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 2
        else:                    # moving to a different route
            for k in range(1, len(rou)):
                rou_test = rou[:k] + [cust, cust_folw1, cust_folw2] + rou[k:]
                if check_violation(rou_test, 1)[0]:
                    adjust_cost2 = check_violation(rou_test, 1)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 1
                elif check_violation(rou_test, 2)[0]:
                    adjust_cost2 = check_violation(rou_test, 2)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new = rou_test
                        move_to_route = j
                        new_type = 2

    if best_cut_cost > 1e-5:
        print ('shift3 good', best_cut_cost)
        sol_in6[move_to_route] = route_new
        sol_type6[move_to_route] = new_type
        if move_to_route != c_loc[0]:                   # moving to a different route
            sol_in6[c_loc[0]] = route_ing
            sol_type6[c_loc[0]] = route_type(route_ing)

    elif sa_lns and best_cut_cost < -1e-5:

        prb = random.uniform(0, 1)
        if np.exp(best_cut_cost / curr_temp) > prb:
            print ('shift3', best_cut_cost)
            sol_in6[move_to_route] = route_new
            sol_type6[move_to_route] = new_type
            if move_to_route != c_loc[0]:               # moving to a different route
                sol_in6[c_loc[0]] = route_ing
                sol_type6[c_loc[0]] = route_type(route_ing)


In [29]:
def exchange_1_cust(sol_in3, cust, c_loc, curr_temp, sol_type3, sa_lns):
    """如果可行的话，交换两个客户的位置(是否相同路线)，看看是否能降低总成本."""

    route_ing = copy.deepcopy(sol_in3[c_loc[0]])

    route_new_1 = route_ing
    route_new_2 = route_ing
    exch_to_route = c_loc[0]
    origin_cost1 = check_violation(route_ing, sol_type3[c_loc[0]])[1]
    # route_ing.remove(cust)  # move c in the current route
    # adjust_cost1 = check_violation(route_ing)[1]
    best_cut_cost0 = -1000
    best_cut_cost = best_cut_cost0            # best cost cut of moving this customer
    for j, rou in enumerate(sol_in3):
        origin_cost2 = check_violation(rou, sol_type3[j])[1]
        if j == c_loc[0]:                      # exchange in the same route
            for k in range(1, len(rou)-1):
                if k == c_loc[1]:
                    continue
                rou_test = copy.deepcopy(sol_in3[c_loc[0]])
                rou_test[k], rou_test[c_loc[1]] = rou_test[c_loc[1]], rou_test[k]
                if check_violation(rou_test, 1)[0]:
                    adjust_cost2 = check_violation(rou_test, 1)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test
                        route_new_2 = rou_test
                        exch_to_route = j

                elif check_violation(rou_test, 2)[0]:
                    adjust_cost2 = check_violation(rou_test, 2)[1]
                    cost_cut_test = origin_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test
                        route_new_2 = rou_test
                        exch_to_route = j

        else:                     # exchange to a different route
            for k in range(1, len(rou)-1):
                rou_test_1 = copy.deepcopy(sol_in3[c_loc[0]])
                rou_test_2 = copy.deepcopy(rou)
                rou_test_1[c_loc[1]] = rou[k]
                rou_test_2[k] = cust
                if check_violation(rou_test_1, 1)[0] and check_violation(rou_test_2, 1)[0]:
                    adjust_cost1 = check_violation(rou_test_1, 1)[1]
                    adjust_cost2 = check_violation(rou_test_2, 1)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test_1
                        route_new_2 = rou_test_2
                        exch_to_route = j

                elif check_violation(rou_test_1, 2)[0] and check_violation(rou_test_2, 2)[0]:
                    adjust_cost1 = check_violation(rou_test_1, 2)[1]
                    adjust_cost2 = check_violation(rou_test_2, 2)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test_1
                        route_new_2 = rou_test_2
                        exch_to_route = j


    if best_cut_cost > 1e-5:
        print ('exchange1 good', best_cut_cost)
        sol_in3[c_loc[0]] = route_new_1
        sol_in3[exch_to_route] = route_new_2
        sol_type3[c_loc[0]] = route_type(route_new_1)
        sol_type3[exch_to_route] = route_type(route_new_2)

    elif sa_lns and best_cut_cost < -1e-5:
        prb = random.uniform(0, 1)
        if np.exp(best_cut_cost / curr_temp) > prb:
            print ('exchange1', best_cut_cost)
            sol_in3[c_loc[0]] = route_new_1
            sol_in3[exch_to_route] = route_new_2
            sol_type3[c_loc[0]] = route_type(route_new_1)
            sol_type3[exch_to_route] = route_type(route_new_2)

    # return sol_in3 

In [30]:
def exchange_2_cust(sol_in4, cust, c_loc, curr_temp, sol_type4, sa_lns):
    """将连续2个客户的位置交换到另外2个客户的位置，看看是否可以降低成本."""

    route_ing = copy.deepcopy(sol_in4[c_loc[0]])
    route_new_1 = route_ing
    route_new_2 = route_ing
    cust_folw = route_ing[c_loc[1] + 1]
    exch_to_route = c_loc[0]
    origin_cost1 = check_violation(route_ing, sol_type4[c_loc[0]])[1]
    # route_ing.remove(cust)  # move c in the current route
    # adjust_cost1 = check_violation(route_ing)[1]
    best_cut_cost0 = -1000
    best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
    for j, rou in enumerate(sol_in4):
        origin_cost2 = check_violation(rou, sol_type4[j])[1]
        if j != c_loc[0] and len(rou) >= 4:  # exchange to a different route
            for k in range(1, len(rou) - 2):
                rou_test_1 = copy.deepcopy(sol_in4[c_loc[0]])
                rou_test_2 = copy.deepcopy(rou)
                rou_test_1[c_loc[1]], rou_test_1[c_loc[1] + 1] = rou[k], rou[k + 1]
                rou_test_2[k], rou_test_2[k + 1] = cust, cust_folw
                if check_violation(rou_test_1, 1)[0] and check_violation(rou_test_2, 1)[0]:
                    adjust_cost1 = check_violation(rou_test_1, 1)[1]
                    adjust_cost2 = check_violation(rou_test_2, 1)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test_1
                        route_new_2 = rou_test_2
                        exch_to_route = j

                if check_violation(rou_test_1, 2)[0] and check_violation(rou_test_2, 2)[0]:
                    adjust_cost1 = check_violation(rou_test_1, 2)[1]
                    adjust_cost2 = check_violation(rou_test_2, 2)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test_1
                        route_new_2 = rou_test_2
                        exch_to_route = j


    if best_cut_cost > 1e-5:
        print ('exchange2 good', best_cut_cost)
        sol_in4[c_loc[0]] = route_new_1
        sol_in4[exch_to_route] = route_new_2
        sol_type4[c_loc[0]] = route_type(route_new_1)
        sol_type4[exch_to_route] = route_type(route_new_2)

    elif sa_lns and best_cut_cost < -1e-5:
        prb = random.uniform(0, 1)
        if np.exp(best_cut_cost / curr_temp) > prb:
            print ('exchange2', best_cut_cost)
            sol_in4[c_loc[0]] = route_new_1
            sol_in4[exch_to_route] = route_new_2
            sol_type4[c_loc[0]] = route_type(route_new_1)
            sol_type4[exch_to_route] = route_type(route_new_2)

    # return sol_in4 

In [31]:
def two_exchange_sol(sol_in5, curr_temp, sol_type5, sa_lns):
    """Two-Exchange operator: For two customers i and j on the same route where i is visited before j,
    remove arcs (i,i+),(j,j+); add arcs (i,j),(i+,j+); and reverse the orientation of the arcs between i+ and j.
    Given a solution, check all possible neighborhood.
    """
    solu = copy.deepcopy(sol_in5)
    best_cut_cost0 = -1000
    best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
    adjust_rou_ind = 0
    route_new = sol_in5[adjust_rou_ind]
    for i, rou in enumerate(solu):
        if len(rou) >= 6:
            origin_cost = check_violation(rou, sol_type5[i])[1]
            for k in range(1, len(rou)-4):
                for l in range(k+3, len(rou)-1):
                    route_test = copy.deepcopy(rou)
                    route_test[k], route_test[l] = route_test[l], route_test[k]
                    route_test[k+1: l] = route_test[l-1:k:-1]  # middle reverse
                    if check_violation(route_test, 1)[0]:
                        adjust_cost = check_violation(route_test, 1)[1]
                        if origin_cost - adjust_cost > best_cut_cost:
                            best_cut_cost = origin_cost - adjust_cost
                            adjust_rou_ind = i
                            route_new = route_test

                    elif check_violation(route_test, 2)[0]:
                        adjust_cost = check_violation(route_test, 2)[1]
                        if origin_cost - adjust_cost > best_cut_cost:
                            best_cut_cost = origin_cost - adjust_cost
                            adjust_rou_ind = i
                            route_new = route_test


    if best_cut_cost > 1e-5:
        print ('2exchange good', best_cut_cost)
        sol_in5[adjust_rou_ind] = route_new
        sol_type5[adjust_rou_ind] = route_type(route_new)

    elif sa_lns and best_cut_cost < -1e-5:
        prb = random.uniform(0, 1)
        if np.exp(best_cut_cost / curr_temp) > prb:
            print ('2exchange', best_cut_cost)
            sol_in5[adjust_rou_ind] = route_new
            sol_type5[adjust_rou_ind] = route_type(route_new)

    # return sol_in5


In [32]:
def two_opt(sol_in7, cust, c_loc, curr_temp, sol_type7, sa_lns):
    """2-opt*: given customer i in route a and customer j in route b, exchange the following sequences of i and j.
    for example, initial route a: ...-i-i1-i2-..., initial route b: ...-j-j1-j2-...
    New route a: ...-i-j1-j2-..., new route b: ...-j-i1-i2-..."""

    route_ing = copy.deepcopy(sol_in7[c_loc[0]])

    route_new_1 = route_ing
    route_new_2 = route_ing
    exch_to_route = c_loc[0]
    origin_cost1 = check_violation(route_ing, sol_type7[c_loc[0]])[1]
    # route_ing.remove(cust)  # move c in the current route
    # adjust_cost1 = check_violation(route_ing)[1]
    best_cut_cost0 = -1000
    best_cut_cost = best_cut_cost0  # best cost cut of moving this customer
    for j, rou in enumerate(sol_in7):
        origin_cost2 = check_violation(rou, sol_type7[j])[1]
        if j != c_loc[0]:  # 2-opt* operator has to be implemented in 2 different routes
            for k in range(1, len(rou) - 1):
                rou_test_1 = sol_in7[c_loc[0]][:c_loc[1]] + rou[k:]
                rou_test_2 = rou[:k] + sol_in7[c_loc[0]][c_loc[1]:]
                if check_violation(rou_test_1, 1)[0] and check_violation(rou_test_2, 1)[0]:
                    adjust_cost1 = check_violation(rou_test_1, 1)[1]
                    adjust_cost2 = check_violation(rou_test_2, 1)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test_1
                        route_new_2 = rou_test_2
                        exch_to_route = j

                elif check_violation(rou_test_1, 2)[0] and check_violation(rou_test_2, 2)[0]:
                    adjust_cost1 = check_violation(rou_test_1, 2)[1]
                    adjust_cost2 = check_violation(rou_test_2, 2)[1]
                    cost_cut_test = origin_cost1 + origin_cost2 - adjust_cost1 - adjust_cost2
                    if cost_cut_test > best_cut_cost:
                        best_cut_cost = cost_cut_test
                        route_new_1 = rou_test_1
                        route_new_2 = rou_test_2
                        exch_to_route = j

    if best_cut_cost > 1e-5:
        print ('2opt* good', best_cut_cost)
        sol_in7[c_loc[0]] = route_new_1
        sol_in7[exch_to_route] = route_new_2
        sol_type7[c_loc[0]] = route_type(route_new_1)
        sol_type7[exch_to_route] = route_type(route_new_2)

    elif sa_lns and best_cut_cost < -1e-5:
        prb = random.uniform(0, 1)
        if np.exp(best_cut_cost / curr_temp) > prb:
            print ('2opt*', best_cut_cost)
            sol_in7[c_loc[0]] = route_new_1
            sol_in7[exch_to_route] = route_new_2
            sol_type7[c_loc[0]] = route_type(route_new_1)
            sol_type7[exch_to_route] = route_type(route_new_2)


In [34]:
def simulated_annealing(sol_in, veh_type_in, cost_in):
    """基于5个操作符的邻域搜索。在每次迭代中，随机选择一个操作符."""

    itr_cost = []
    solu = copy.deepcopy(sol_in)
    solu_type = copy.deepcopy(veh_type_in)
    best_solu = sol_in
    best_val = cost_in
    tabu_list = []
    random.seed(10)
    itr = 0
    temp = initial_temp
    t_run = time.time()
    while temp > stop_temp and t_run - t_str < max_run_time:
        itr += 1
        print (itr)
        if itr >= 6000:
            sa_lns = True  # use sa or lns
        else:
            sa_lns = False
        c = random.randint(1, cust_num - 1)  # randomly generated moving customer
        while c in tabu_list:
            c = random.randint(1, cust_num - 1)
        c_loc = cust_loc(solu, c)

        if len(solu[c_loc[0]]) < 4:  
            # customer number less than 2, can only implement shift1 and exchange1 operator
            wheel_value1 = random.uniform(0, 1)
            if wheel_value1 < 0.45:
                shift_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
            elif wheel_value1 < 0.9:
                exchange_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
            else:
                two_opt(solu, c, c_loc, temp, solu_type, sa_lns)
        elif len(solu[c_loc[0]]) >= 4 and c_loc[1] <= len(solu[c_loc[0]]) - 3:  
            # customer number more than 2, can implement all operators
            wheel_value2 = random.uniform(0, 1)
            if wheel_value2 < 0.2:
                shift_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
            elif wheel_value2 < 0.4:
                shift_2_cust(solu, c, c_loc, temp, solu_type, sa_lns)
            elif wheel_value2 < 0.6:
                exchange_1_cust(solu, c, c_loc, temp, solu_type, sa_lns)
            elif wheel_value2 < 0.8:
                exchange_2_cust(solu, c, c_loc, temp, solu_type, sa_lns)
            else:
                two_opt(solu, c, c_loc, temp, solu_type, sa_lns)


        if itr % 150 == 0:  # implement two-exchange operator every 200 iteration
            two_exchange_sol(solu, temp, solu_type, sa_lns)


        temp -= delta
        tabu_list.append(c)
        if len(tabu_list) > 200:
            tabu_list.pop(0)

        cost_i = get_cost(solu, solu_type, False)
        itr_cost.append(cost_i)
        if cost_i < best_val:
            best_solu = solu
            best_val = cost_i


        t_run = time.time()

    # Adjust0: delete consecutive depots(0)
    adjust_sol0 = []
    for route0 in best_solu:
        if len(route0) <= 2:  # [0, 0] route
            continue
        elif len(route0) == 3 and cust_need[route0[1]][0] == 4:  # [0, charge, 0] route
            continue
        else:
            new_route0 = [0]
            for j in range(1, len(route0)):
                if new_route0[-1] == 0 and route0[j] == 0:  # delete consecutive depots
                    continue
                elif cust_need[new_route0[-1]][0] == 4 and cust_need[route0[j]][0] == 4:  # delete consecutive charges
                    continue
                else:
                    new_route0.append(route0[j])
            adjust_sol0.append(new_route0)

    # 调整1:删除最后一个充电站，如果不需要
    adjust_sol1 = []
    for route1 in adjust_sol0:
        if cust_need[route1[-2]][0] == 4:  # 最后一个是充电站，判断是否需要这个充电站
            type1 = route_type(route1)
            route1_cost = check_violation(route1, type1)[1]
            veh_rout_test = copy.deepcopy(route1)
            veh_rout_test.pop(-2)
            if check_violation(veh_rout_test, 1)[0]:
                if check_violation(veh_rout_test, 1)[1] < route1_cost:
                    adjust_sol1.append(veh_rout_test)
                else:
                    adjust_sol1.append(route1)
            elif check_violation(veh_rout_test, 2)[0]:
                if check_violation(veh_rout_test, 2)[1] < route1_cost:
                    adjust_sol1.append(veh_rout_test)
                else:
                    adjust_sol1.append(route1)
            else:
                adjust_sol1.append(route1)
        else:
            adjust_sol1.append(route1)



    return adjust_sol1, itr_cost


In [37]:
if __name__ == '__main__':
    t0 = time.time()
    max_run_time = 300.
    file_num = '1_1601'
    # files = ['1_1601', '2_1501', '3_1401', '4_1301', '5_1201']
    # nn_para = [[0.64, 0.23, 0.13], [0.63, 0.23, 0.14], [0.65, 0.2, 0.15], [0.63, 0.23, 0.14], [0.63, 0.23, 0.14]]
    small_veh_para = [255, 240, 210, 190, 190]
    # cost_tt = []
    # for f_i, file_num in enumerate(files):
    t_str = time.time()
    print ('We are running data inputnode_%s' % file_num)
    file_code = int(file_num[0])
    charge_num = 100  # number of charge station
    cust_num = int(file_num[-4:]) - charge_num  # including depot, delivery and pickup
    pickup = 200
    print ('Total customer number including depot: ', cust_num)
    cust_need, time_mat, dist_mat = read_data(file_num)
    cust_charge = []  # the nearest charge station from customer, including depot
    for i in range(cust_num):
        charg_dist = copy.deepcopy(dist_mat[i, :])
        charg_dist[1:cust_num] = 100000
        min_dist = np.where(charg_dist == np.min(charg_dist))
        near_id = min_dist[0][0]
        cust_charge.append(near_id)

    iveco_cap = [1, 2.0, 12, 100000, 0.012, 200]  # [veh_type, weight, volume, max_distance, trs_cost, fix_cost]
    truck_cap = [2, 2.5, 16, 120000, 0.014, 300]  # [veh_type, weight, volume, max_distance, trs_cost, fix_cost]
    charg_cost0 = 0.5 * 100  # charge cost
    wait_cost0 = 24.0  # waiting cost: 24 yuan/h
    charg_t = 30  # charge time at charge station
    depot_t = 60  # stay time at depot after every round
    oper_t = 30  # operation time at each customer
    start_t = 60 * 8  # earliest start time of a vehicle
    alp, bet, gam = 0.64, 0.23, 0.13  # weight of t_ij, h_ij and v_ij of Time-oriented Nearest-Neighbor
    # alp, bet, gam = nn_para[f_i]
    output = lns_initial(small_veh=255)
    veh_typ = vehicle_type_adjust(output)
    cost = get_cost(solution=output, veh_type=veh_typ, if_write=False)
    print ('\nTotal used vehicle number: ', len(output))
    print ('Total cost is: ', cost)

    # oprt1, oprt2, oprt3, oprt4, oprt5, oprt7 = 0, 0, 0, 0, 0, 0
    initial_temp, stop_temp, delta = 40., 10., 30./6000
    new_output, cost_t = simulated_annealing(sol_in=output, veh_type_in=veh_typ, cost_in=cost)
    veh_typ1 = vehicle_type_adjust(new_output)
    new_cost = get_cost(solution=new_output, veh_type=veh_typ1, if_write=False)
    print ('\nTotal used vehicle number: ', len(new_output))
    print ('Total cost is: ', new_cost)
    # cost_tt.append(cost_t)
    # break
    t1 = time.time()
    print ('Total elapsed time: ', t1 - t0)

    plt.plot(cost_t)
    plt.show()
    # plt.plot(cost_tt[0])
    # plt.show()
    # plt.plot(cost_tt[1])
    # plt.show()
    # plt.plot(cost_tt[2])
    # plt.show()
    # plt.plot(cost_tt[3])
    # plt.show()
    # plt.plot(cost_tt[4])
    # plt.show()
    # new_cost = get_cost([[0, 660, 200, 926, 782, 0, 1256, 0]], [2], False)
    # print new_cost
    # print '\nTotal used vehicle number: ', len(new_output)
    # print 'Total cost is: ', new_cost 

We are running data inputnode_1_1601
Total customer number including depot:  1501


TypeError: not all arguments converted during string formatting